Benedict Greenwood benedict.greenwood AT ucl.ac.uk

Script combines and cleans data from empathy task, in which two subjects (on "participant" and one "companion") received electrocutaneous shocks at one of three intensities (high/low/safe). Participants also underwent a resting state psychophysiology recording.

This script links task datafiles and psychophysiology datafiles from all participants, cleans/recodes the data, supports visual data checks, and calculates psychophysiology parameters.

Import packages

In [ ]:
# =========================
# Standard library imports
# =========================
import os  
import glob
import time
import datetime
import re
from pathlib import Path


# =========================
# Third-party core libraries
# =========================
import numpy as np
import pandas as pd


# =========================
# File I/O / MATLAB / HDF5
# =========================
import h5py  # for importing from .mat file
from scipy.io import savemat  # for saving as .mat file


# =========================
# Signal processing / scientific computing
# =========================
from scipy.signal import detrend, medfilt
from scipy.interpolate import interp1d, CubicSpline


# =========================
# Neurophysiology / ECG / HRV toolkits
# =========================
import neurokit2 as nk
import systole as sys
from systole.detection import ecg_peaks  # for detecting peaks
from systole.detection import ecg_peaks  # for detecting peaks (duplicate import)
import rapidhrv as rhv
import heartpy as hp


# =========================
# Spike2 / CED / Neo handling
# =========================
import sonpy
from sonpy import lib as sp
from neo.io.cedio import CedIO
from neo.core import Event
from quantities import Hz, s


# =========================
# Visualization libraries
# =========================
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]  # Set standard plotsize

import seaborn as sns

import bokeh as bk
bk.io.output_notebook()  # Start Bokeh

import plotly.graph_objs as go
import plotly.express as px
import kaleido


Loading BokehJS ...

Define durations

In [ ]:
#Important - define duration constants (in ms)
DURATION_PRE_PHASE = 2000 #duration to include before start of section (for baseline)
DURATION_WAITING = 3000 #waiting phase
DURATION_SHOCK = 500 #shock duration
DURATION_RESPONSE_EXCL_SHOCK = 6000 #response phase excluding shock
SPIKE_FS = 1000 #spike sampling rate in Hz

Get list of session folders

In [ ]:
BASE_DIR = Path(r"F:/ADHD emotion/Empathy study")
directory = BASE_DIR / "Participant data"

# Check if directory exists
if not os.path.isdir(directory):
    raise FileNotFoundError(f"Directory not found: {directory}")


# Find folders for each session
initialsToFind = 'BG'
session_folders = [folder for folder in os.listdir(directory) 
              if os.path.isdir(os.path.join(directory, folder)) and folder.startswith(initialsToFind)]

gorillaDataFolder = os.path.join(directory, 'HCT gorilla data')

# Print the total count and list of folders
print(f"/nTotal number of 'BG' folders containing participant data: {len(session_folders)}")
for folder in session_folders:
    print(folder)

# Define print statement colours
RED = '\033[91m'
BLUE = '\033[94m'
RESET = '\033[0m'  # Reset to default color

/nTotal number of 'BG' folders containing participant data: 50
BG001
BG002
BG003
BG004
BG005
BG006
BG007
BG008
BG009
BG010
BG011
BG012
BG013
BG014
BG015
BG016
BG017
BG018
BG019
BG020
BG021
BG022
BG023
BG024
BG025
BG026
BG027
BG028
BG029
BG030
BG031
BG032
BG033
BG034
BG035
BG036
BG037
BG038
BG039
BG040
BG041
BG042
BG043
BG044
BG045
BG046
BG047
BG048
BG049
BG050


Import matlab task data for all participants

In [ ]:
# Create empty dataframe to load each participant's data into
df_all_matlab = pd.DataFrame()

print('--------Extracting MATLAB task data----------')

# Dictionary of manually scored faulty trials
manual_faulty_trials = {
    'BG001': [8, 41, 42, 76],
    'BG002': [7, 27],
    'BG003': [36, 47],
    'BG004': [21, 42],
    'BG005': [6, 30, 42],
    'BG006': [43, 44],
    'BG007': [3, 18, 36, 61, 67]
}

for sessionID in session_folders:     

    # get session folder
    sessionFolder = os.path.join(directory, sessionID)
    print('Session: ', sessionID)
    
    # Import matlab (task) data files
    matlab_csv_datafile = os.path.join(sessionFolder, sessionID + "_empathy.csv")
    matlab_search_pattern = os.path.join(sessionFolder, f'*{matlab_csv_datafile}')

    try:
        df_matlab = pd.read_csv(matlab_csv_datafile)
    except:
        print('Error importing CSV file containing MATLAB data')

    # If there is no column indexing faulty trials then add one
    if 'faultyShock' not in df_matlab.columns:
        df_matlab['faultyShock'] = 0

    # Score faulty trials using the dictionary
    faultyTrialNos = manual_faulty_trials.get(sessionID, [])
    if faultyTrialNos:
        print('-', 'Manually scored faulty trials')

    rowsToChange = df_matlab['trialNo'].isin(faultyTrialNos)
    df_matlab.loc[rowsToChange, 'faultyShock'] = 1

    # IMPORTANT - exclude final trial from BG021 due to Spike crashing
    if sessionID == "BG021":
        trialToExclude = [80]
        df_matlab = df_matlab[~df_matlab['trialNo'].isin(trialToExclude)]
        print('- Manually deleted trial 80 due to Spike crashing')

    # Add this session's data to df_all_matlab
    df_all_matlab = pd.concat([df_all_matlab, df_matlab])
    print('-', 'Extracted MATLAB data')

# Rename the column 'sujectID' to 'subjectID' in df_all_matlab
df_all_matlab.rename(columns={'sujectID': 'subjectID'}, inplace=True)

# Save df_all_matlab
df_all_matlab.to_csv('Data checks/df_all_matlab.csv')

--------Extracting matlab task data----------
Session:  BG001
- Manually scored faulty trials
- Extracted matlab data
Session:  BG002
- Manually scored faulty trials
- Extracted matlab data
Session:  BG003
- Manually scored faulty trials
- Extracted matlab data
Session:  BG004
- Manually scored faulty trials
- Extracted matlab data
Session:  BG005
- Manually scored faulty trials
- Extracted matlab data
Session:  BG006
- Manually scored faulty trials
- Extracted matlab data
Session:  BG007
- Manually scored faulty trials
- Extracted matlab data
Session:  BG008
- Extracted matlab data
Session:  BG009
- Extracted matlab data
Session:  BG010
- Extracted matlab data
Session:  BG011
- Extracted matlab data
Session:  BG012
- Extracted matlab data
Session:  BG013
- Extracted matlab data
Session:  BG014
- Extracted matlab data
Session:  BG015
- Extracted matlab data
Session:  BG016
- Extracted matlab data
Session:  BG017
- Extracted matlab data
Session:  BG018
- Extracted matlab data
Session:  

Import tablet ratings data for all participants

In [ ]:
#Create empty dataframe to load each participant's data into
df_all_ratings = pd.DataFrame()

print('--------Extracting tablet ratings----------')

for sessionID in session_folders:     

    # get session folder
    sessionFolder = os.path.join(directory, sessionID)
    print('Session: ', sessionID)
    
    # Import tablet data files for PARTICIPANT
    print('-', 'Extracting', sessionID, 'p', 'ratings data')
    p_search_pattern = sessionID + 'p'
    year_search_pattern = '2024'
    p_search_regex = re.compile(f'.*{p_search_pattern}.*{year_search_pattern}.*\.csv', re.IGNORECASE)  # create regex search pattern that ignores case
    p_all_csv_files = glob.glob(os.path.join(sessionFolder, '*.csv'))  # Use glob to find all CSV files
    p_ratings_files = [f for f in p_all_csv_files if p_search_regex.match(os.path.basename(f))]

    if len(p_ratings_files) == 1:
        #if there is one ratings file then extract data
        p_ratings_file = p_ratings_files[0]
        df_p_ratings = pd.read_csv(p_ratings_file, skiprows=1) #import as dataframe
        df_all_ratings = pd.concat([df_all_ratings, df_p_ratings]) #add this session's data to df_all_ratings

    elif len(p_ratings_files) == 0:
        print(RED + '-', 'No ratings file found' + RESET)

    elif len(p_ratings_files) > 1:
        print(RED + '-', len(p_ratings_files), 'ratings file found' + RESET)

        #*IMPORTANT - process subjects with multiple datafiles (i.e. when participant accidentally quit so we had to re-open tablet ratings html)
        if p_search_pattern == 'BG004p':
            
            df_p_ratings = pd.DataFrame() #create empty df

            for i_file in range(len(p_ratings_files)):
                print(BLUE + '-', 'Processing CSV file no. ', i_file, RESET)

                p_ratings_file = p_ratings_files[i_file]
                p_ratings = pd.read_csv(p_ratings_file, skiprows=1) #import as dataframe

                if i_file == 1:
                    df_p_ratings = pd.concat([df_p_ratings, p_ratings]) #add this session's data to df_all_rating
                    max_trial_no_prev_file = p_ratings['trialNo'].iloc[-1] #calculate maximum trial number this file
                    
                elif i_file == 2: #rescore trial numbers
                    p_ratings['trialNo'] = p_ratings['trialNo'] + max_trial_no_prev_file #take max trial number from previous file and add it to trial numbers in curent file 
                    df_p_ratings = pd.concat([df_p_ratings, p_ratings]) #add this session's data to df_all_rating
        
        df_all_ratings = pd.concat([df_all_ratings, df_p_ratings]) #add this session's data to df_all_ratings

    # Import matlab (task) data files for COMPANION
    print('-', 'Extracting', sessionID, 'c', 'ratings data')
    c_search_pattern = sessionID + 'c'
    c_search_regex = re.compile(f'.*{c_search_pattern}.*{year_search_pattern}.*\.csv', re.IGNORECASE)  # create regex search pattern that ignores case
    c_all_csv_files = glob.glob(os.path.join(sessionFolder, '*.csv'))  # Use glob to find all CSV files
    c_ratings_files = [f for f in c_all_csv_files if c_search_regex.match(os.path.basename(f))]
    

    if len(c_ratings_files) == 1:
        #if there is one ratings file then extract
        c_ratings_file = c_ratings_files[0]
        df_c_ratings = pd.read_csv(c_ratings_file, skiprows=1) #import as dataframe
        df_all_ratings = pd.concat([df_all_ratings, df_c_ratings]) #add this session's data to df_all_ratings

    elif len(c_ratings_files) == 0:
        print(RED + '-', 'No ratings file found' + RESET)

    elif len(c_ratings_files) > 1:
        print(RED + '-', len(c_ratings_files), 'ratings file found' + RESET)

#save df_all_ratings
df_all_ratings.to_csv('Data checks/df_all_ratings.csv')

--------Extracting tablet ratings----------
Session:  BG001
- Extracting BG001 p ratings data
- Extracting BG001 c ratings data
Session:  BG002
- Extracting BG002 p ratings data
- Extracting BG002 c ratings data
Session:  BG003
- Extracting BG003 p ratings data
- Extracting BG003 c ratings data
Session:  BG004
- Extracting BG004 p ratings data
- 3 ratings file found
- Processing CSV file no.  0 
- Processing CSV file no.  1 
- Processing CSV file no.  2 
- Extracting BG004 c ratings data
Session:  BG005
- Extracting BG005 p ratings data
- Extracting BG005 c ratings data
Session:  BG006
- Extracting BG006 p ratings data
- Extracting BG006 c ratings data
Session:  BG007
- Extracting BG007 p ratings data
- Extracting BG007 c ratings data
Session:  BG008
- Extracting BG008 p ratings data
- Extracting BG008 c ratings data
Session:  BG009
- Extracting BG009 p ratings data
- Extracting BG009 c ratings data
Session:  BG010
- Extracting BG010 p ratings data
- Extracting BG010 c ratings data
Ses

Link ratings data to task data

In [34]:
#function to insert rows

def insert_rows(ratings, trial_no_insert_after, count_new_rows, subjID):
    insert_index = ratings[ratings['trialNo'] == trial_no_insert_after].index[0] + 1 #get index to insert at
    new_rows = pd.DataFrame({ #define new rows
        'userID': [subjID] * count_new_rows,
        'phase': ['main_task'] * count_new_rows
    }, index=range(count_new_rows))

    ratings = pd.concat([ #add new rows
        ratings.iloc[:insert_index],
        new_rows,
        ratings.iloc[insert_index:]
    ]).reset_index(drop=True)

    return ratings


In [ ]:
# Create a dataframe with matlab task data to add ratings data to
df_all_combined = df_all_matlab.copy()

# define variables to add as nans
varsToAdd = ['sensationLocation', 'sensationCoordinates', 'sensationCoordinatesPerc', 'sensationIntensity', 'ratingArousal', 'ratingValence', 'emotionIntensity']

print('--------Linking tablet ratings to matlab task data----------')

# Dictionary mapping participant IDs to trial numbers to delete
trial_nos_dict = {
    # PARTICIPANT IDs
    'BG001p': [1,2,3,4,5,6,7,8,9,10,11,14,57],
    'BG003p': [21],
    'BG007p': [5],
    'BG008p': [10],
    'BG009p': [17, 18],
    'BG011p': [7],
    'BG014p': [39, 40],
    'BG016p': [34],
    'BG018p': [3, 4, 11],
    'BG022p': [7],
    'BG026p': [3,8,9,11,12,13,18],
    'BG028p': [7],
    'BG030p': [3, 9, 42],
    'BG041p': [34],
    'BG044p': [3, 9, 10],
    'BG046p': [6],
    'BG049p': [8, 14],

    # COMPANION IDs
    'BG001c': [1,2,3,4,5,6,7,8,9,10,11],
    'BG004c': [5, 22, 24],
    'BG007c': [5],
    'BG008c': [8],
    'BG009c': [17, 18],
    'BG017c': [40],
    'BG026c': [3,8,9,11,12,13,18],
    'BG028c': [7],
    'BG029c': [8, 35],
    'BG039c': [3],
    'BG044c': [3,8,9,44],
    'BG046c': [2],
    'BG049c': [13]
}

for sessionID in session_folders:

    print('Session: ', sessionID)

    # start p_rows_inserted as False
    p_rows_inserted = False
    c_rows_inserted = False

    # -------------Link data from PARTICIPANT
    p_id = sessionID + 'p'
    print('-', 'Processing', p_id)
    
    # Find relevant trials (rows) in df_all_matlab
    p_rows_combined = df_all_combined['subjectID'].str.contains((p_id), na=False, case=False)
    rating_rows_combined = df_all_combined['ratingSubmitted'] == 'yes'
    p_rating_rows_combined = (p_rows_combined) & (rating_rows_combined)

    # Find relevant trials (rows) in df_all_ratings
    p_rows_allratings = df_all_ratings['userID'].str.contains((p_id), na=False, case=False)
    p_ratings = df_all_ratings[p_rows_allratings]

    # set trial_nos_to_delete using dictionary
    trial_nos_ratings_to_delete = trial_nos_dict.get(p_id, [])

    # Special participant-specific corrections
    if p_id == 'BG001p':
        p_ratings.loc[:, 'phase'] = 'main_task'
    elif p_id == 'BG004p':
        insert_after_trial = 23
        n_new_rows = 4
        p_ratings = insert_rows(p_ratings, insert_after_trial, n_new_rows, p_id)
        p_rows_inserted = True
    elif p_id == 'BG006p':
        p_ratings.loc[p_ratings['trialNo'] == 30, 'shockRecipient'] = 'Self'
    elif p_id == 'BG009p':
        p_ratings.loc[p_ratings['trialNo'] == 39, 'shockRecipient'] = 'Self'
    elif p_id == 'BG011p':
        p_ratings.loc[p_ratings['trialNo'] == 21, 'shockRecipient'] = 'Other'
    elif p_id == 'BG017p':
        p_ratings.loc[p_ratings['trialNo'] == 19, 'shockRecipient'] = 'Self'
    elif p_id == 'BG021p':
        p_ratings.loc[p_ratings['trialNo'] == 33, 'shockRecipient'] = 'Other'
    elif p_id == 'BG022p':
        p_ratings = p_ratings.iloc[1:43]
    elif p_id == 'BG024p':
        p_ratings = p_ratings.iloc[1:42]
    elif p_id == 'BG027p':
        p_ratings.loc[p_ratings['trialNo'] == 29, 'shockRecipient'] = 'Self'
    elif p_id == 'BG028p':
        p_ratings = p_ratings.iloc[1:44]
    elif p_id == 'BG029p':
        p_rows_inserted = True
    elif p_id == 'BG030p':
        insert_after_trial = 38
        n_new_rows = 1
        p_ratings = insert_rows(p_ratings, insert_after_trial, n_new_rows, p_id)
        p_rows_inserted = True
    elif p_id == 'BG035p':
        p_ratings = p_ratings.iloc[1:42]
        p_rows_inserted = True
    elif p_id == 'BG036p':
        p_rows_inserted = True
    elif p_id == 'BG037p':
        p_ratings.loc[p_ratings['trialNo'] == 35, 'shockRecipient'] = 'Self'
    elif p_id == 'BG044p':
        p_ratings.loc[p_ratings['trialNo'] == 25, 'shockRecipient'] = 'Other'
    elif p_id == 'BG045p':
        p_ratings.loc[p_ratings['trialNo'] == 5, 'shockRecipient'] = 'Self'
    elif p_id == 'BG046p':
        insert_after_trial = 1
        n_new_rows = 1
        p_ratings = insert_rows(p_ratings, insert_after_trial, n_new_rows, p_id)
        p_rows_inserted = True
    elif p_id == 'BG047p':
        p_ratings.loc[p_ratings['trialNo'] == 29, 'shockRecipient'] = 'Self'
    elif p_id == 'BG049p':
        insert_after_trial = 18
        n_new_rows = 1
        p_ratings = insert_rows(p_ratings, insert_after_trial, n_new_rows, p_id)
        p_rows_inserted = True
    elif p_id == 'BG050p':
        p_ratings = p_ratings.iloc[1:42]

    # delete rows
    p_rows = p_ratings['userID'].str.contains((p_id), na=False, case=False)
    p_rows_to_delete = (p_ratings['trialNo'].isin(trial_nos_ratings_to_delete)) & (p_rows)
    p_ratings = p_ratings[~p_rows_to_delete]

    # Find relevant trials (rows) in df_all_ratings now ratings have been fixed
    p_rows_main_task = (p_ratings['phase'] == 'main_task')
    p_rows = p_ratings['userID'].str.contains((p_id), na=False, case=False)
    p_rows_ratings_maintask = (p_rows) & (p_rows_main_task)
    p_ratings_to_keep = p_ratings.loc[p_rows_ratings_maintask, varsToAdd].reset_index(drop=True)

    p_conds_tablet_debugging = p_ratings.loc[p_rows_ratings_maintask, 'shockRecipient'].str.lower().reset_index(drop=True)
    p_conds_matlab_debugging = df_all_combined.loc[p_rating_rows_combined, 'shockRecipient'].reset_index(drop=True)

    if sum(p_rating_rows_combined) != len(p_ratings_to_keep):
        print(RED + '-', 'ERROR: there are', sum(p_rating_rows_combined), 'rows in matlab data requiring ratings, but', len(p_ratings_to_keep), 'rows of ratings in tablet data' + RESET)
    elif (p_rows_inserted == False) & (not p_conds_tablet_debugging.equals(p_conds_matlab_debugging)):
        print(RED + '-' , 'ERROR: Mismatches between tablet shock recipients and matlab' + RESET)
    else:
        df_all_combined.loc[p_rating_rows_combined, varsToAdd] = p_ratings_to_keep[varsToAdd].values
        print(BLUE + '-', 'Successfully linked ratings to matlab task data' + RESET)

    # -------------Link data from COMPANION
    c_id = sessionID + 'c'
    print('-', 'Processing', c_id)

    c_rows_combined = df_all_combined['subjectID'].str.contains((c_id), na=False, case=False)
    rating_rows_combined = df_all_combined['ratingSubmitted'] == 'yes'
    c_rating_rows_combined = (c_rows_combined) & (rating_rows_combined)

    c_rows_allratings = df_all_ratings['userID'].str.contains((c_id), na=False, case=False)
    c_ratings = df_all_ratings[c_rows_allratings]

    trial_nos_ratings_to_delete = trial_nos_dict.get(c_id, [])

    # Special companion-specific corrections
    if c_id == 'BG001c':
        c_ratings.loc[:, 'phase'] = 'main_task'
        c_ratings.loc[c_ratings['trialNo'] == 47, 'shockRecipient'] = 'Other'
    elif c_id == 'BG004c':
        insert_after_trial = 16
        n_new_rows = 1
        c_ratings = insert_rows(c_ratings, insert_after_trial, n_new_rows, c_id)
        c_rows_inserted = True
    elif c_id == 'BG010c':
        c_ratings.loc[c_ratings['trialNo'] == 23, 'shockRecipient'] = 'Other'
    elif c_id == 'BG011c':
        c_rows_inserted = True
    elif c_id == 'BG016c':
        c_ratings.loc[c_ratings['trialNo'] == 19, 'shockRecipient'] = 'Self'
    elif c_id == 'BG018c':
        c_rows_inserted = True
    elif c_id == 'BG019c':
        c_rows_inserted = True
    elif c_id == 'BG022c':
        c_rows_inserted = True
    elif c_id == 'BG027c':
        c_ratings.loc[c_ratings['trialNo'] == 24, 'shockRecipient'] = 'Self'
    elif c_id == 'BG029c':
        c_ratings.loc[c_ratings['trialNo'] == 38, 'shockRecipient'] = 'Self'
    elif c_id == 'BG030c':
        c_rows_inserted = True
    elif c_id == 'BG037c':
        c_ratings.loc[c_ratings['trialNo'] == 4, 'shockRecipient'] = 'Self'
    elif c_id == 'BG041c':
        c_ratings.loc[c_ratings['trialNo'] == 29, 'shockRecipient'] = 'Other'
    elif c_id == 'BG046c':
        insert_after_trial = 5
        n_new_rows = 1
        c_ratings = insert_rows(c_ratings, insert_after_trial, n_new_rows, c_id)
        c_rows_inserted = True

    # delete rows
    c_rows = c_ratings['userID'].str.contains((c_id), na=False, case=False)
    c_rows_to_delete = (c_ratings['trialNo'].isin(trial_nos_ratings_to_delete)) & (c_rows)
    c_ratings = c_ratings[~c_rows_to_delete]

    c_rows_main_task = (c_ratings['phase'] == 'main_task')
    c_rows = c_ratings['userID'].str.contains((c_id), na=False, case=False)
    c_rows_ratings_maintask = (c_rows) & (c_rows_main_task)
    c_ratings_to_keep = c_ratings.loc[c_rows_ratings_maintask, varsToAdd].reset_index(drop=True)

    c_conds_tablet_debugging = c_ratings.loc[c_rows_ratings_maintask, 'shockRecipient'].str.lower().reset_index(drop=True)
    c_conds_matlab_debugging = df_all_combined.loc[c_rating_rows_combined, 'shockRecipient'].reset_index(drop=True)

    if sum(c_rating_rows_combined) != len(c_ratings_to_keep):
        print(RED + '-', 'ERROR: there are', sum(c_rating_rows_combined), 'rows in matlab data requiring ratings, but', len(c_ratings_to_keep), 'rows of ratings in tablet data' + RESET)
    elif (c_rows_inserted == False) & (not c_conds_tablet_debugging.equals(c_conds_matlab_debugging)):
        print(RED + '-' , 'ERROR: Mismatches between tablet shock recipients and matlab' + RESET)
    else:
        df_all_combined.loc[c_rating_rows_combined, varsToAdd] = c_ratings_to_keep[varsToAdd].values
        print(BLUE + '-', 'Successfully linked ratings to matlab task data' + RESET)


--------Linking tablet ratings to matlab task data----------
Session:  BG001
- Processing BG001p
- Successfully linked ratings to matlab task data
- Processing BG001c
- Successfully linked ratings to matlab task data
Session:  BG002
- Processing BG002p
- Successfully linked ratings to matlab task data
- Processing BG002c
- Successfully linked ratings to matlab task data
Session:  BG003
- Processing BG003p
- Successfully linked ratings to matlab task data
- Processing BG003c
- Successfully linked ratings to matlab task data
Session:  BG004
- Processing BG004p
- Successfully linked ratings to matlab task data
- Processing BG004c
- Successfully linked ratings to matlab task data
Session:  BG005
- Processing BG005p
- Successfully linked ratings to matlab task data
- Processing BG005c
- Successfully linked ratings to matlab task data
Session:  BG006
- Processing BG006p
- Successfully linked ratings to matlab task data
- Processing BG006c
- Successfully linked ratings to matlab task data
Ses

Amend ratings based on testing session notes

In [ ]:
print('--------Editing data in line with testing notes----------')

# Path to the Excel file
trials_to_edit_file = = BASE_DIR / "Participant data" / "Data checks" / "notes_trials_to_edit.xlsx"

try:
    # Import the Excel file into a DataFrame
    df_trials_to_edit = pd.read_excel(trials_to_edit_file)
    print("Excel file imported successfully.")
except Exception as e:
    print('Error importing xlsx file:', e)
    exit()

# Iterate through rows in the DataFrame
for index, row in df_trials_to_edit.iterrows():
    # Extract the subject ID from the current row
    subjID = row['SubjectID']  # Use column name directly from the DataFrame
    print('Subject: ', subjID)
    
    # Find participant trials (rows) in df_all_combined
    subj_rows_combined = df_all_combined['subjectID'].str.contains(subjID, na=False, case=False)  # Ensure correct column name

    # Rescore faulty trials
    if pd.notna(row['trialsFaultyShock']):  # Check if 'trialsFaultyShock' is not NA
        faultyTrials = row['trialsFaultyShock']

        # Identify rows to amend
        rows_to_amend = (df_all_combined['trialNo'] == faultyTrials) & (subj_rows_combined)

        # Update the 'faultyShock' column
        df_all_combined.loc[rows_to_amend, 'faultyShock'] = 1

        print(' - Scored faulty trials')

    # Update low shock strength for self
    if pd.notna(row['newLowSelf']): 
        df_all_combined.loc[subj_rows_combined, 'lowIntensitySelf'] = row['newLowSelf']
        print(' - Updated lowIntensitySelf')

    # Update high shock strength for self
    if pd.notna(row['newHighSelf']): 
        df_all_combined.loc[subj_rows_combined, 'highIntensitySelf'] = row['newHighSelf']
        print(' - Updated highIntensitySelf')

    # Update low shock strength for other
    if pd.notna(row['newLowOther']): 
        df_all_combined.loc[subj_rows_combined, 'lowIntensityOther'] = row['newLowOther']
        print(' - Updated lowIntensityOther')

    # Update high shock strength for other
    if pd.notna(row['newHighOther']): 
        df_all_combined.loc[subj_rows_combined, 'highIntensityOther'] = row['newHighOther']
        print(' - Updated highIntensityOther')

#BG039 - change trial 10 to NOT an error trial.
# Find subject trials (rows) in df_all_combined
subj_rows_combined = df_all_combined['subjectID'].str.contains('BG0039p|BG0039c', na=False, case=False)
rows_to_amend = (df_all_combined['trialNo'] == 10) & (subj_rows_combined)
df_all_combined.loc[rows_to_amend, 'faultyShock'] = 0
print('Subjects BG039p + BG039c')
print(' - Scored faulty trial')

#save df_all_ratings for debugging
#saveFileCombined = BASE_DIR / "Participant data" / "Preprocessed task data" / "Empathy task" / "df_all_combined.csv"
#df_all_combined.to_csv(saveFileCombined)

--------Editing data in line with testing notes----------
Excel file imported successfully.
Subject:  BG001p
 - Scored faulty trials
Subject:  BG001c
 - Scored faulty trials
Subject:  BG002p
 - Scored faulty trials
Subject:  BG002c
 - Scored faulty trials
Subject:  BG003p
 - Scored faulty trials
Subject:  BG003c
 - Scored faulty trials
Subject:  BG004p
 - Scored faulty trials
Subject:  BG004c
 - Scored faulty trials
Subject:  BG005p
 - Updated lowIntensityOther
Subject:  BG005c
 - Updated lowIntensitySelf
Subject:  BG006p
 - Scored faulty trials
Subject:  BG006c
 - Scored faulty trials
Subject:  BG007p
 - Scored faulty trials
Subject:  BG007c
 - Scored faulty trials
Subject:  BG009p
 - Updated lowIntensitySelf
 - Updated highIntensitySelf
 - Updated highIntensityOther
Subject:  BG009c
 - Updated highIntensitySelf
 - Updated lowIntensityOther
 - Updated highIntensityOther
Subject:  BG022p
 - Scored faulty trials
Subject:  BG022c
 - Scored faulty trials
Subject:  BG026p
 - Scored faulty 

Fcuntion for determinig breakpoints in Spike file

In [ ]:
def divide_into_n_breakpoints(number, n):
    """
    Split an integer length into n approximately equal breakpoints.

    Example:
        number=10, n=3 -> [0, 4, 7, 10]
    """
    
    base = number // n
    remainder = number % n
    breakpoints = [0]  # Start with 0
    
    for i in range(1, n):
        point = i * base + min(i, remainder)
        breakpoints.append(point)
    
    breakpoints.append(number)  # End with the original number
    
    return breakpoints

Function for plotting ECG peaks

In [ ]:
def plot_ECG_peaks(sessionID, p_or_c, ecg_data, ecg_peaks, ecg_signal, df_textmarks, resting_or_main):

    """
    Plot ECG signal with detected peaks and flagged outlier inter-beat intervals (IBIs).

    If `resting_or_main` is "resting", the ECG is cropped using resting-state textmarks ("rss", "rse").
    If "main", selected task textmarks ("apr", "rat") are added as annotated vertical lines.

    Args:
        sessionID (str): Session identifier.
        p_or_c (str): "p" for participant or "c" for companion.
        ecg_data: Object containing ECG time values in `ecg_data.times`.
        ecg_peaks (array-like of bool): Boolean mask indicating detected peaks.
        ecg_signal (array-like): ECG amplitude values.
        df_textmarks (pd.DataFrame): DataFrame with 'text' and 'sampleNo' columns.
        resting_or_main (str): Either "resting" or "main".

    Returns:
        None: Displays an interactive Plotly figure.

    Notes:
        Requires `SPIKE_FS` (and optionally `RED`) to be defined globally.
    """
    
    fig = go.Figure()
    print(sessionID)

    #extract ecg times
    ecg_times = ecg_data.times

    if resting_or_main == 'resting':
        #Find textmarks for start and end of resting state recording
        if sessionID == 'BG002':
            #Process exceptions
            tm_rs_start = 418900
            tm_rs_end = tm_rs_start + 120000

        #For all othr sessions
        else:
            tm_rs_start = int(df_textmarks.loc[df_textmarks['text'] == 'rss', 'sampleNo'].iloc[-1]) # take final rss textmark
            tm_rs_end = int(df_textmarks.loc[df_textmarks['text'] == 'rse', 'sampleNo'].iloc[-1]) # take final rss textmark
        
        #Check textmarks found
        if pd.isna(tm_rs_start):
            print(RED + 'WARNING: no rss textmark found')
        elif pd.isna(tm_rs_end):
            print(RED + 'WARNING: no rse textmark found')
            
        ecg_times = ecg_data.times[tm_rs_start : tm_rs_end]
        ecg_signal = ecg_signal[tm_rs_start : tm_rs_end]
        ecg_peaks = ecg_peaks[tm_rs_start : tm_rs_end]

    #Create peak massk for plot
    peak_mask = ecg_peaks == True # Create a mask for true values in p_ecg_peaks

    #Identify outlier IBIs based on being much shorter or longer than previous ibi
    peak_indices = np.where(peak_mask)[0]
    ibis = np.diff(peak_indices)
    outlier_ibi_mask = np.zeros_like(ibis, dtype=bool)
    for i in range(1, len(ibis)):
        if ibis[i] < 0.6 * ibis[i-1] or ibis[i] > 1.4 * ibis[i-1]:
            outlier_ibi_mask[i] = True

    full_outlier_mask = np.zeros_like(peak_mask, dtype=bool) # Create a full-length mask for outliers
    full_outlier_mask[peak_indices[1:][outlier_ibi_mask]] = True

    # Add the time series data
    fig.add_trace(go.Scatter(x=ecg_times, y=ecg_signal, mode='lines', name='ECG Signal')) #plot ecg trace

    fig.add_trace(go.Scatter(x=ecg_times[peak_mask], #plot peaks
        y=np.ones(sum(peak_mask)),  # All y-values are set to 1
        mode='markers',
        marker=dict(color='red', size=8, symbol='cross'),  # Customize marker style
        name='ECG Peaks'
    ))

    fig.add_trace(go.Scatter(x=ecg_times[full_outlier_mask], #plot outlier ibis
        y=np.ones(sum(full_outlier_mask)),  # All y-values are set to 1
        mode='markers',
        marker=dict(color='yellow', size=8, symbol='cross'),  # Customize marker style
        name='Outliers'
    ))

    ## Add textmarks
    if resting_or_main == 'main':
        textmarks_of_interest = ['apr', 'rat'] #apr = anticipation before name displayed; rat = subjects prompted to submit rating
        df_textmarks_subset = df_textmarks[df_textmarks['text'].isin(textmarks_of_interest)]
        for _, row in df_textmarks_subset.iterrows():
            tm_sample_no = row['sampleNo']
            tm_text = row['text']
            
            # Add vertical line
            fig.add_vline(x=tm_sample_no/(SPIKE_FS), line_width=2, line_dash="dash", line_color="grey") #divide sample number by SPIKE_FS to convert to s
            
            # Add text annotation
            fig.add_annotation(
                x=tm_sample_no/(SPIKE_FS),
                y=1,  # Adjust this value to position the text vertically
                text=tm_text,
                showarrow=False,
                textangle=-90,
                yanchor="bottom",
                font=dict(color="black", size=20)
            )
        
    #Add title specifying p or c
    if p_or_c == 'p':
        part_type = 'participant'
    else:
        part_type = 'companion'

    fig.update_layout(
        title = f"{sessionID}: {part_type} data"
    )
    
    fig.show()

Functions for calculating HR responses

In [ ]:

def calculate_mean_hr(start_idx, end_idx, inst_hr_timeseries, subj_noisy_samples, SPIKE_FS):
    """
    Compute mean instantaneous heart rate (HR) over a sample window.

    Returns NaN if any noisy samples occur within the window.

    Args:
        start_idx (int): Start sample index (inclusive).
        end_idx (int): End sample index (exclusive).
        inst_hr_timeseries (array-like): Instantaneous HR values (bpm) per sample.
        subj_noisy_samples (array-like of bool): Mask indicating noisy samples.
        SPIKE_FS (int or float): Sampling frequency (Hz). Included for compatibility.

    Returns:
        float: Mean HR (bpm) or np.nan if window contains noise.
    """
    # Exclude window if any noisy samples
    if np.any(subj_noisy_samples[start_idx:end_idx]):
        return np.nan
    
    else:
        mean_hr = inst_hr_timeseries[start_idx:end_idx].mean()
        return mean_hr


def downsample_linear_interpolation(data, orig_fs, target_fs):
    """
    Downsample a signal to a target sampling rate using linear interpolation.

    Args:
        data (array-like): Input signal.
        orig_fs (float): Original sampling rate (Hz).
        target_fs (float): Target sampling rate (Hz).

    Returns:
        np.ndarray: Downsampled signal.
    """
    # Calculate the duration of the signal
    duration = len(data) / orig_fs

    # Create the original time points
    original_time = np.linspace(0, duration, len(data), endpoint=True)

    # Create the new time points for the target frequency
    num_target_samples = int(round(duration * target_fs) + 1) #add 1 to ensure last sample is included
    target_time = np.linspace(0, duration, num_target_samples, endpoint=True)

    # Perform linear interpolation
    downsampled_data = np.interp(target_time, original_time, data)
    
    return downsampled_data


def downsample_with_last(arr, original_fs, target_fs):
    """
    Downsample a signal by integer decimation while ensuring the final sample is included.

    Args:
        arr (array-like): Input signal.
        original_fs (float): Original sampling rate (Hz).
        target_fs (float): Target sampling rate (Hz).

    Returns:
        np.ndarray: Downsampled signal including the last sample.
    """
    factor = int(original_fs / target_fs)

    idx = np.arange(0, len(arr), factor)
    if idx[-1] != len(arr) - 1:
        idx = np.append(idx, len(arr) - 1)
    return arr[idx]


def calculate_hr_timeseries(start_sample, end_sample, inst_hr_timeseries, noisy_samples, orig_fs, target_fs):
    """
    Extract and downsample instantaneous HR for a given window, excluding noisy segments.

    Returns an array containing NaN if noise is present in the window.

    Args:
        start_sample (int): Start sample index (inclusive).
        end_sample (int): End sample index (inclusive).
        inst_hr_timeseries (array-like): Instantaneous HR values (bpm) per sample.
        noisy_samples (array-like of bool): Mask indicating noisy samples.
        orig_fs (float): Original sampling rate (Hz).
        target_fs (float): Target sampling rate (Hz).

    Returns:
        np.ndarray: Downsampled HR series (rounded to 2 decimals) or [np.nan].
    """
    # Check for noisy samples in phase
    if np.any(noisy_samples[start_sample:end_sample+1]):
        return np.array([np.nan])
    else:
        # Downsample to target frequency and include last sample
        downsampled_hr = np.round(downsample_with_last(inst_hr_timeseries[start_sample:end_sample], orig_fs, target_fs), 2)

    return downsampled_hr


def calculate_instantaneous_hr(subj_ecg_peaks, sampling_rate):
    """
    Compute instantaneous heart rate (bpm) from ECG R-peak locations.

    HR is assigned to all samples between consecutive peaks. Output is the same
    length as the input signal, with NaNs where HR cannot be computed.

    Args:
        subj_ecg_peaks (array-like of bool): Boolean mask indicating R-peak samples.
        sampling_rate (float): ECG sampling rate (Hz).

    Returns:
        np.ndarray: Instantaneous HR time series (bpm).
    """
    subj_ecg_peaks = np.asarray(subj_ecg_peaks)
    peak_indices = np.where(subj_ecg_peaks)[0]
    hr_timeseries = np.full_like(subj_ecg_peaks, np.nan, dtype=float)

    if len(peak_indices) < 2:
        return hr_timeseries  # Not enough peaks to compute HR

    # Calculate RR intervals and HR for each interval
    rr_intervals = np.diff(peak_indices) / sampling_rate
    hr_values = 60.0 / rr_intervals

    # Assign HR value to all samples between each pair of R-peaks
    for i in range(len(rr_intervals)):
        start = peak_indices[i]
        end = peak_indices[i+1]
        hr_timeseries[start:end] = hr_values[i]

    return hr_timeseries


def interpolate_ectopic_hr(inst_hr_timeseries, p_ecg_peaks, idx_p_ectopics_to_interpolate):
    """
    Interpolate HR values around ectopic peaks using cubic spline interpolation.

    For each ectopic peak, HR values between the nearest surrounding true R-peaks
    are replaced with spline-interpolated values.

    Args:
        inst_hr_timeseries (array-like): Instantaneous HR values (bpm) per sample.
        p_ecg_peaks (array-like of bool): Boolean mask indicating detected peaks.
        idx_p_ectopics_to_interpolate (array-like of int): Indices of ectopic peaks.

    Returns:
        np.ndarray: HR time series with ectopic intervals interpolated.
    """
    inst_hr_timeseries_interp = inst_hr_timeseries.copy()
    r_peak_indices = np.where(p_ecg_peaks)[0]

    # Exclude ectopic peaks from true R-peaks
    true_r_peaks = np.setdiff1d(r_peak_indices, idx_p_ectopics_to_interpolate)

    # Identify all indices to interpolate (between true peaks surrounding each ectopic)
    for ectopic_idx in idx_p_ectopics_to_interpolate:
        prev_peaks = true_r_peaks[true_r_peaks < ectopic_idx]
        next_peaks = true_r_peaks[true_r_peaks > ectopic_idx]
        if len(prev_peaks) == 0 or len(next_peaks) == 0:
            print(RED + f"Cannot interpolate for ectopic peak at index {ectopic_idx}: no bounding true peaks." + RESET)
            continue  # Cannot interpolate if no bounding true peaks
        
        # Get the last true peak before the ectopic and the first true peak after
        prev_peak = prev_peaks[-1]
        next_peak = next_peaks[0]

        #extend sample numbers by 2 to make sure we do not include ectopic HR
        prev_peak = prev_peak - 2
        next_peak = next_peak + 2

        # Indices to interpolate (exclude endpoints)
        interp_indices = np.arange(prev_peak, next_peak)
        
        # Known points (start and end of the segment)
        x_known = np.array([prev_peak, next_peak])
        y_known_hr = inst_hr_timeseries_interp[x_known]
        
        # Create cubic spline interpolator
        cs_hr = CubicSpline(x_known, y_known_hr)

        # Interpolate and replace hr values in inst_hr_timeseries_interp
        inst_hr_timeseries_interp[interp_indices] = cs_hr(interp_indices)

    return inst_hr_timeseries_interp


ECG data checks and HR response calculations

In [ ]:
run_ecg_analyses = True #define whether to run this section of skip it
hr_visual_verification = False #whether to plot HR trace

if run_ecg_analyses:
    # Suppress the specific DeprecationWarning
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    # Visual verification variables
    num_plotted = 0
    max_plotted = 100
    ecg_visual_verif_done = [] #Add session ID here once visual verification done

    #------- Peak detection algorithm exceptions
    peak_detection_algs = {sessionID: 'sleepecg' for sessionID in session_folders}
    #peak_detection_algs['BG026'] = 'hamilton' #option to swap peak detection algorithm to 'hamilton' for selected participants if 'sleepecg' algorithm

    #------- Import excel file containing notes from ecg inspection
    analysisFolder = BASE_DIR / "Analysis"
    ecg_inspection_file = os.path.join(analysisFolder, 'Data checks', 'empathy_ecg_inspection.xlsx')
    df_ecg_inspection = pd.read_excel(ecg_inspection_file)

    #Predefine hr timeseries columns as objects to allow insertion of entire timeseries into each cell
    df_all_combined['hr_series_waiting'] = None
    df_all_combined = df_all_combined.astype({'hr_series_waiting': 'object'})

    df_all_combined['hr_series_anticipation'] = None
    df_all_combined = df_all_combined.astype({'hr_series_anticipation': 'object'})

    df_all_combined['hr_series_response'] = None
    df_all_combined = df_all_combined.astype({'hr_series_response': 'object'})

    for sessionID in session_folders:

        # get session folder
        sessionFolder = os.path.join(directory, sessionID)
        print('Session: ', sessionID) 
        
        # Import Spike2 data file
        spike_file_extension = "empathy.smrx"
        spike_search_pattern = os.path.join(sessionFolder, f'*{spike_file_extension}')
        spike_files = glob.glob(spike_search_pattern)
        if len(spike_files) > 1:
            print("-", "{} SMRX files found.".format(len(spike_files)))
        elif len(spike_files) < 1:
            print(RED + 'WARNING: no empathy spike data files found')

        # Loop over files
        for i_spike_file in range(len(spike_files)):
            
            spike_filename = spike_files[i_spike_file]

            print("-", "Importing Spike data for {}".format(sessionID))

            # Read Spike2 file
            reader = CedIO(filename=spike_filename)
            
            block = reader.read_block()
            spike2data = block.segments[0] # single segment

            # Create a dictionary of analog channels and their indexes
            analog_channel_dictionary = {}

            for data_channel in range(len(spike2data.analogsignals)):
                channel_idx = data_channel
                channel_name = spike2data.analogsignals[data_channel].array_annotations['channel_names'][0]
                analog_channel_dictionary[channel_name] = channel_idx

            # ---------- Get the ECG data for the subjects
            p_ecg_data = spike2data.analogsignals[analog_channel_dictionary['p_ECG']] #*IMPORTANT
            c_ecg_data = spike2data.analogsignals[analog_channel_dictionary['c_ECG']] #*IMPORTANT
        
        # ---------- Extract textmarks from smrx file using SonPy
        smrfile = sonpy.lib.SonFile(spike_filename, True)
        textmarkChanNo = 30
        chanIdxInFile = textmarkChanNo - 1
        max_n_tick = smrfile.ChannelMaxTime(chanIdxInFile)

        n_chunks = 4
        breakpoints = divide_into_n_breakpoints(max_n_tick, n_chunks)
        df_textmarks_allchunks = pd.DataFrame()

        for i_chunk in range(n_chunks):
            first_sample = breakpoints[i_chunk]
            last_sample = breakpoints[i_chunk + 1]

            # Adjust the last chunk to ensure it includes the final tick
            if i_chunk == n_chunks - 1:
                last_sample += 1  # Add 1 to include the very last tick
            
            nSamplesToImport = last_sample - first_sample
            textmarks = smrfile.ReadTextMarks(chan=chanIdxInFile, nMax=nSamplesToImport, tFrom=first_sample, tUpto=last_sample)

            textmarks_sampleNo = np.full(len(textmarks), np.nan)
            textmarks_text = np.array(['---'] * len(textmarks), dtype=str)

            for i_mark in range(len(textmarks)):
                mark = textmarks[i_mark]
                textmarks_sampleNo[i_mark] = round(mark.Tick / 100)
                textmarks_text[i_mark] = ''.join([str(mark[0]), str(mark[1]), str(mark[2])])
            
            df_textmarks = pd.DataFrame({'sampleNo': textmarks_sampleNo, 'text': textmarks_text})
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, df_textmarks], ignore_index=True)
        
        
        # Get details of ecg data
        p_ecg_data_np = np.squeeze(p_ecg_data)
        c_ecg_data_np = np.squeeze(c_ecg_data)
        p_ecg_signal = p_ecg_data.magnitude.squeeze()
        c_ecg_signal = c_ecg_data.magnitude.squeeze()
        fps_p_ecg = int(p_ecg_data.sampling_rate.magnitude)
        fps_c_ecg = int(c_ecg_data.sampling_rate.magnitude)

        #*IMPORTANT - Process exceptions
        if sessionID == "BG004":
            p_ecg_data_np.magnitude[3933300:3975500] = 0
            p_ecg_signal[3933300:3975500] = 0

        if sessionID == "BG026":
            p_ecg_data_np.magnitude[0:343500] = 0
            p_ecg_signal[0:343500] = 0

        # Detect R peaks *IMPORTANT* lines
        _, p_ecg_peaks = ecg_peaks(signal=p_ecg_data_np, method='sleepecg', sfreq=fps_p_ecg) # participant data - use sleepecg algorithm from Systole
        _, c_ecg_peaks = ecg_peaks(signal=c_ecg_data_np, method='sleepecg', sfreq=fps_c_ecg) # companion data - use sleepecg algorithm from Systole


        if hr_visual_verification and sessionID not in ecg_visual_verif_done and num_plotted < max_plotted:
            plot_ECG_peaks(sessionID, 'p', p_ecg_data, p_ecg_peaks, p_ecg_signal, df_textmarks_allchunks, resting_or_main = 'main')
            plot_ECG_peaks(sessionID, 'c', c_ecg_data, c_ecg_peaks, c_ecg_signal, df_textmarks_allchunks, resting_or_main = 'main')
        
        #------Participant - Delete entire sections of mislabelled peaks-------
        print(sessionID + 'p' + ' peak deletion/addition')

        # Find rows in ecg inspection excel file for this recording from this participant
        row_p_recording = (df_ecg_inspection['pID'] == sessionID + 'p')

        # Delete entire sections of peaks (for replacing slightly but acceptably noisy sections with specific peaks listed in add_peaks)
        p_idx_sections_to_delete_peaks = df_ecg_inspection.loc[row_p_recording, 'sections_all_peaks_zero'] # Get sample numbers of entire sections of peaks to delete

        if isinstance(p_idx_sections_to_delete_peaks, str): # Ensure it's a list or iterable of strings, in case it's a single value
            p_idx_sections_to_delete_peaks = [p_idx_sections_to_delete_peaks]

        p_indices_to_delete = [] # Initialise an empty list to collect all indices to delete

        for cell_value in p_idx_sections_to_delete_peaks: # Process each cell value
            if cell_value.lower() == 'none':  # Skip cells with 'none'
                continue
            for section in cell_value.split(','):  # Split multiple ranges by commas
                start, end = map(int, section.split(':'))  # Split each range and convert to integers
                p_indices_to_delete.extend(range(start, end + 1))  # Include the endpoint

        if p_indices_to_delete: # Convert to a numpy array for assignment if there are indices to delete
            p_indices_to_delete = np.array(p_indices_to_delete, dtype=int)
            # Delete all peaks in the specified sections
            p_ecg_peaks[p_indices_to_delete] = False  # Set the values to False
            print(
                BLUE + '-',
                'Successfully deleted peaks from entire ranges:',
                ', '.join(p_idx_sections_to_delete_peaks),
                RESET
            )


        #------Participant - Delete specific mislabelled peaks-------
        # Get sample numbers of peaks to delete
        idx_p_peaks_to_delete = df_ecg_inspection.loc[row_p_recording, 'delete_peaks']
        idx_p_peaks_to_delete = idx_p_peaks_to_delete.to_numpy() #convert from series to numpy
        idx_p_peaks_to_delete = np.fromstring(','.join(idx_p_peaks_to_delete), sep=',', dtype=float)#convert to float
        idx_p_peaks_to_delete = idx_p_peaks_to_delete.astype(int)

        if len(idx_p_peaks_to_delete) == 0:
            print('-', 'No peaks deleted')

        else:
            #check that there is a peak at this sample number (i.e. that the value in p_ecg_peaks at that sample is 1)
            p_peak_vals = p_ecg_peaks[idx_p_peaks_to_delete]
            p_mislabelled_peaks = idx_p_peaks_to_delete[p_peak_vals == 0] #get sample numbers where there is no peak to delete
            p_truelabelled_peaks = idx_p_peaks_to_delete[p_peak_vals == 1] #get sample numbers where there is indeed a peak to delete

            #Delete peaks
            if len(p_truelabelled_peaks) > 0:
                p_ecg_peaks[p_truelabelled_peaks] = False #change from one to zero in p_ecg_peaks_all_chunks to delete peak
                print(BLUE + '-', 'Successfully deleted peaks at',  str(p_truelabelled_peaks) + RESET)

            # Print warning if there is no peak at the sample no. then print warning
            if len(p_mislabelled_peaks) > 0:
                print(RED + 'ERROR: No peaks to delete at sample nos. ', p_mislabelled_peaks, RESET) #print message

                #For some participants the sample numbers are shifted by 1 so check if there are peaks on the previous samples
                p_peak_vals_min1 = p_ecg_peaks[idx_p_peaks_to_delete - 1]
                p_mislabelled_peaks_min1 = idx_p_peaks_to_delete[p_peak_vals_min1 == 0]
                #or next samples
                p_peak_vals_plus1 = p_ecg_peaks[idx_p_peaks_to_delete + 1]
                p_mislabelled_peaks_plus1 = idx_p_peaks_to_delete[p_peak_vals_plus1 == 0]

                if any(p_peak_vals_min1) == 1:
                    #If all peaks are present 1 sample previously then process
                    print(BLUE + 'However peaks found on previous samples ' + str(idx_p_peaks_to_delete[p_peak_vals_min1 == 1]-1) + RESET) 

                    if len(p_mislabelled_peaks_min1 > 0 ):
                        print(RED + 'However no peaks found at ' + str(p_mislabelled_peaks_min1-1) + RESET)

                    #delete peaks from previous samples
                    #p_ecg_peaks[idx_p_peaks_to_delete[p_peak_vals_min1 == 1]-1] = 0 #change from one to zero in p_ecg_peaks_all_chunks to delete peak
                    print(BLUE + '-', 'Successfully deleted peaks from previous samples at',  str(idx_p_peaks_to_delete[p_peak_vals_min1 == 1]-1) + RESET)
                    

                if any(p_peak_vals_plus1) == 1:
                    #If any peaks are present 1 sample subsequently then process
                    print(BLUE + 'However peaks found on next samples ' + str(idx_p_peaks_to_delete[p_peak_vals_plus1 == 1]) + RESET) 

                    if len(p_mislabelled_peaks_plus1 > 0 ):
                        print(RED + 'However no peaks found at ' + str(p_mislabelled_peaks_plus1) + RESET)

                    #delete peaks from subsequent samples
                    #p_ecg_peaks[idx_p_peaks_to_delete[p_peak_vals_plus1 == 1]] = 0 #change from one to zero in p_ecg_peaks_all_chunks to delete peak
                    print(BLUE + '-', 'Successfully deleted peaks from subsequent samples at',  str(idx_p_peaks_to_delete[p_peak_vals_plus1 == 1]) + RESET)

            #Delete peaks on sample, previous sample, and next sample to be sure
            p_ecg_peaks[idx_p_peaks_to_delete] = False
            p_ecg_peaks[idx_p_peaks_to_delete - 1] = False
            p_ecg_peaks[idx_p_peaks_to_delete + 1] = False

        #-----------Participant - Add missed peaks---------------------
        # Get sample numbers of peaks to add
        idx_p_peaks_to_add = df_ecg_inspection.loc[row_p_recording, 'add_peaks']
        idx_p_peaks_to_add = idx_p_peaks_to_add.to_numpy() #convert from series to numpy
        idx_p_peaks_to_add = np.fromstring(','.join(idx_p_peaks_to_add), sep=',', dtype=float)#convert to float
        idx_p_peaks_to_add = idx_p_peaks_to_add.astype(int)

        #add peak to p_ecg_peaks
        if len(idx_p_peaks_to_add) > 0:
            p_ecg_peaks[idx_p_peaks_to_add] = 1 #change to 1 in p_ecg_peaks
            print(BLUE + '-', 'Added missing peaks ', str(idx_p_peaks_to_add) + RESET)

        #----------Participant - calculate instantaneous HR---------------------
        #Calculate instantaneous HR from subj_ecg_peaks
        p_inst_hr = calculate_instantaneous_hr(p_ecg_peaks, SPIKE_FS)

        #-----------Participant - interpolate HR for R-R intervals containing ectopic beats---------------------
        #Interpolate location of ectopic beats, but only for ectopics that increase the IBI,other ectopics are just deleted
        # Get sample numbers of approximate positions to add beat
        idx_p_ectopics_to_interpolate = df_ecg_inspection.loc[row_p_recording, 'peak_to_interpolate']
        idx_p_ectopics_to_interpolate = idx_p_ectopics_to_interpolate.to_numpy()
        idx_p_ectopics_to_interpolate = np.fromstring(','.join(idx_p_ectopics_to_interpolate), sep=',', dtype=float)
        idx_p_ectopics_to_interpolate = idx_p_ectopics_to_interpolate.astype(int)

        #Delete ectopic beats
        if len(idx_p_ectopics_to_interpolate) > 0:
            p_ecg_peaks[idx_p_ectopics_to_interpolate] = False
            p_ecg_peaks[idx_p_ectopics_to_interpolate -1] = False
            p_ecg_peaks[idx_p_ectopics_to_interpolate +1] = False

        # Interpolate HR for R-R intervals containing ectopic beats
        if len(idx_p_ectopics_to_interpolate) > 0:
            p_inst_hr = interpolate_ectopic_hr(p_inst_hr, p_ecg_peaks, idx_p_ectopics_to_interpolate)
            print(BLUE + '-', 'Interpolated HR for ', str(len(idx_p_ectopics_to_interpolate)), ' ectopic beats at', str(idx_p_ectopics_to_interpolate) + RESET)

        #------companion - Delete entire section of mislabelled peaks-------
        print(sessionID + 'c' + ' peak deletion/addition')

        # Find rows in ecg inspection excel file for this recording from this companion
        row_c_recording = (df_ecg_inspection['pID'] == sessionID + 'c')

        # Delete entire sections of peaks (for replacing slightly but acceptably noisy sections with specific peaks listed in add_peaks)
        c_idx_sections_to_delete_peaks = df_ecg_inspection.loc[row_c_recording, 'sections_all_peaks_zero'] # Get sample numbers of entire sections of peaks to delete

        if isinstance(c_idx_sections_to_delete_peaks, str): # Ensure it's a list or iterable of strings, in case it's a single value
            c_idx_sections_to_delete_peaks = [c_idx_sections_to_delete_peaks]

        c_indices_to_delete = [] # Initialise an empty list to collect all indices to delete

        for cell_value in c_idx_sections_to_delete_peaks: # Process each cell value
            if cell_value.lower() == 'none':  # Skip cells with 'none'
                continue
            for section in cell_value.split(','):  # Split multiple ranges by commas
                start, end = map(int, section.split(':'))  # Split each range and convert to integers
                c_indices_to_delete.extend(range(start, end + 1))  # Include the endpoint

        if c_indices_to_delete: # Convert to a numpy array for assignment if there are indices to delete
            c_indices_to_delete = np.array(c_indices_to_delete, dtype=int)
            # Delete all peaks in the specified sections
            c_ecg_peaks[c_indices_to_delete] = False  # Set the values to zero
            print(
                BLUE + '-',
                'Successfully deleted peaks from entire ranges:',
                ', '.join(c_idx_sections_to_delete_peaks),
                RESET
            )
        
        #------Companion - Delete specific mislabelled peaks-------
        # Get sample numbers of peaks to delete
        idx_c_peaks_to_delete = df_ecg_inspection.loc[row_c_recording, 'delete_peaks']
        idx_c_peaks_to_delete = idx_c_peaks_to_delete.to_numpy() #convert from series to numpy
        idx_c_peaks_to_delete = np.fromstring(','.join(idx_c_peaks_to_delete), sep=',', dtype=float)#convert to float
        idx_c_peaks_to_delete = idx_c_peaks_to_delete.astype(int)

        if len(idx_c_peaks_to_delete) == 0:
            print('-', 'No peaks deleted')

        else:
            #check that there is a peak at this sample number (i.e. that the value in c_ecg_peaks at that sample is 1)
            c_peak_vals = c_ecg_peaks[idx_c_peaks_to_delete]
            c_mislabelled_peaks = idx_c_peaks_to_delete[c_peak_vals == 0] #get sample numbers where there is no peak to delete
            c_truelabelled_peaks = idx_c_peaks_to_delete[c_peak_vals == 1] #get sample numbers where there is indeed a peak to delete

            #Delete peaks
            if len(c_truelabelled_peaks) > 0:
                #c_ecg_peaks[c_truelabelled_peaks] = 0 #change from one to zero in c_ecg_peaks_all_chunks to delete peak
                print(BLUE + '-', 'Successfully deleted peaks at',  str(c_truelabelled_peaks) + RESET)

            # Print warning if there is no peak at the sample no. then print warning
            if len(c_mislabelled_peaks) > 0:
                print(RED + 'ERROR: No peaks to delete at sample nos. ', c_mislabelled_peaks, RESET) #print message

                #For some participants the sample numbers are shifted by 1 so check if there are peaks on the previous samples
                c_peak_vals_min1 = c_ecg_peaks[idx_c_peaks_to_delete - 1]
                c_mislabelled_peaks_min1 = idx_c_peaks_to_delete[c_peak_vals_min1 == 0]
                #or next samples
                c_peak_vals_plus1 = c_ecg_peaks[idx_c_peaks_to_delete + 1]
                c_mislabelled_peaks_plus1 = idx_c_peaks_to_delete[c_peak_vals_plus1 == 0]

                if any(c_peak_vals_min1) == 1:
                    #If any peaks are present 1 sample previously then process
                    print(BLUE + 'However peaks found on previous samples ' + str(idx_c_peaks_to_delete[c_peak_vals_min1 == 1]-1) + RESET) 


                if any(c_peak_vals_plus1) == 1:
                    #If any peaks are present 1 sample subsequently then process
                    print(BLUE + 'However peaks found on next samples ' + str(idx_c_peaks_to_delete[c_peak_vals_plus1 == 1]) + RESET) 

                    if len(c_mislabelled_peaks_plus1 > 0 ):
                        print(RED + 'However no peaks found at ' + str(c_mislabelled_peaks_plus1) + RESET)

            #Delete peaks on sample, previous sample, and next sample to be sure
            c_ecg_peaks[idx_c_peaks_to_delete] = False
            c_ecg_peaks[idx_c_peaks_to_delete - 1] = False
            c_ecg_peaks[idx_c_peaks_to_delete + 1] = False

        #-----------companion - Add missed peaks---------------------
        # Get sample numbers of peaks to add
        idx_c_peaks_to_add = df_ecg_inspection.loc[row_c_recording, 'add_peaks']
        idx_c_peaks_to_add = idx_c_peaks_to_add.to_numpy() #convert from series to numpy
        idx_c_peaks_to_add = np.fromstring(','.join(idx_c_peaks_to_add), sep=',', dtype=float)#convert to float
        idx_c_peaks_to_add = idx_c_peaks_to_add.astype(int)

        #add peak to c_ecg_peaks_all_chunks
        if len(idx_c_peaks_to_add) > 0:
            c_ecg_peaks[idx_c_peaks_to_add] = True #change to 1 in c_ecg_peaks_all_chunks
            print(BLUE + '-', 'Added missing peaks ', str(idx_c_peaks_to_add) + RESET)
        
        #----------Companion - calculate instantaneous HR---------------------
        #Calculate instantaneous HR from subj_ecg_peaks
        c_inst_hr = calculate_instantaneous_hr(c_ecg_peaks, SPIKE_FS)

        #-----------Participant - Interpolate ectopic beats---------------------
        #Interpolate location of ectopic beats, but only for ectopics that increase the IBI,other ectopics are just deleted
        # Get sample numbers of approximate positions to add beat
        idx_c_ectopics_to_interpolate = df_ecg_inspection.loc[row_c_recording, 'peak_to_interpolate']
        idx_c_ectopics_to_interpolate = idx_c_ectopics_to_interpolate.to_numpy()
        idx_c_ectopics_to_interpolate = np.fromstring(','.join(idx_c_ectopics_to_interpolate), sep=',', dtype=float)
        idx_c_ectopics_to_interpolate = idx_c_ectopics_to_interpolate.astype(int)

        # Interpolate HR for R-R intervals containing ectopic beats
        if len(idx_c_ectopics_to_interpolate) > 0:
            c_inst_hr = interpolate_ectopic_hr(c_inst_hr, c_ecg_peaks, idx_c_ectopics_to_interpolate)
            print(BLUE + '-', 'Interpolated HR for ', str(len(idx_c_ectopics_to_interpolate)), ' ectopic beats at', str(idx_c_ectopics_to_interpolate) + RESET)
        
        #IMPORTANT - process exceptions
        if sessionID == 'BG002':
            #start from row idx 19
            df_textmarks_allchunks = df_textmarks_allchunks.iloc[19:]

        elif sessionID == 'BG021':
            #add rat textmark at last sample of spike file (when it crashed)
            new_row_rating = pd.DataFrame({"sampleNo": [5509836], "text": ['rat']})  # Create a new DataFrame with the new row
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, new_row_rating], ignore_index=True)  # Append the row  
        
        #generate IDs of both subjects in session
        subjsInSession = ["p", "c"]

        for subj in subjsInSession:
            
            subjID = sessionID + subj

            print(subjID + ' calculating HR responses')

            #check that number of anticipation and shock textmarks matches number of trials
            nTrialsInTextmarks = len(df_textmarks_allchunks[df_textmarks_allchunks['text'].isin(['apr'])]) #get number of trials in textmarks
            subj_rows_combined = df_all_combined['subjectID'].str.contains(subjID, na=False, case=False)  #get subject rows in df_all_combined

            if nTrialsInTextmarks != sum(subj_rows_combined):
                print(RED + 'ERROR: there are ' + str(nTrialsInTextmarks) + 
                    ' apr textmarks but ' + str(sum(subj_rows_combined)) + 
                    ' apr textmarks but ' + str(sum(subj_rows_combined)) + 
                    ' rows of data for this subject in df_all_combined' + RESET)
            else:
                print(BLUE + '- No. of apr textmarks (' + str(nTrialsInTextmarks) + 
                    ') matches no. rows (' + str(sum(subj_rows_combined)) + 
                    ') data for this subject in df_all_combined' + RESET)
        
            
            #create array specifying noisy sections
            row_subj_recording = (df_ecg_inspection['pID'] == subjID)# Find rows in ecg inspection excel file for this subject
            idx_noisy_section = df_ecg_inspection.loc[row_subj_recording, 'bad_segments'] # Get sample numbers of entire sections of peaks to delete

            if isinstance(idx_noisy_section, str): # Ensure it's a list or iterable of strings, in case it's a single value
                idx_noisy_section = [idx_noisy_section]

            idx_noisy_trials = [] # Initialise an empty list to collect all indices to delete

            for cell_value in idx_noisy_section: # Process each cell value
                if cell_value.lower() == 'none':  # Skip cells with 'none'
                    continue
                for section in cell_value.split(','):  # Split multiple ranges by commas
                    print(BLUE + '- Processing noisy sections' + RESET)
                    start, end = map(int, section.split(':'))  # Split each range and convert to integers
                    idx_noisy_trials.extend(range(start, end + 1))  # Include the endpoint

            if subj == "p": #IMPORTANT - defines array for calculating HR responses
                subj_ecg_peaks = p_ecg_peaks
                subj_inst_hr = p_inst_hr
            elif subj == "c":
                subj_ecg_peaks = c_ecg_peaks
                subj_inst_hr = c_inst_hr
            else:
                print(RED + 'ERROR - issue defining subj_ecg_peaks - subj not recognised')

            #Process exceptions
            if subjID == 'BG006p':
                #BG006 p and c psychophys channels are wrong way round for empathy task. Swap over!
                subj_ecg_peaks = c_ecg_peaks
                subj_inst_hr = c_inst_hr
            elif subjID == 'BG006c':
                 subj_ecg_peaks = p_ecg_peaks
                 subj_inst_hr = p_inst_hr

            subj_noisy_samples = np.full(len(subj_ecg_peaks), False, dtype=bool) #define logical array
            subj_noisy_samples[idx_noisy_trials] = True #set noisy trials to true

            # Iterate through each trial and calculate mean HR and mean HR change in each section
            #> Filter rows for the relevant textmarks in df_textmarks_all_chunks
            relevant_textmarks = ["apr", #anticipation pre-name
                                  "apo", #anticipation post-name
                                  "sph", "spl", "sps", "sch", "scl", "scs", #shock delivery
                                  "res", #start of response period
                                  "rat"] #submit rating/experimenter change shock settings
            filtered_textmarks = df_textmarks_allchunks[df_textmarks_allchunks["text"].isin(relevant_textmarks)]

            # Find all indices of the "apr" textmark (one for each trial)
            apr_indices = filtered_textmarks[filtered_textmarks["text"] == "apr"].index

            # Iterate through each trial using "apr" textmarks as the starting point
            for trial_idx, apr_idx in enumerate(apr_indices):
                
                #Calculate trial number (start at 1 not 0)
                trial_no = trial_idx + 1
                
                #calculate subject row as logical array
                subject_row = (df_all_combined['subjectID'].str.contains(subjID, na=False, case=False)) & (df_all_combined['trialNo'] == trial_no)

                # Get the indices of the textmarks for this trial
                sample_no_apr = int(filtered_textmarks.loc[apr_idx, "sampleNo"])

                # Get the next "apo" textmark (signalling start of anticipation post name period) after "apr"
                sample_no_apo = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx) & (filtered_textmarks["text"] == "apo"), "sampleNo"
                ].iloc[0]

                # Get the first shock (sph, spl, sps, sch, scl, scs) textmark after "apo"
                sample_no_shock = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx)
                    & (filtered_textmarks["text"].isin(["sph", "spl", "sps", "sch", "scl", "scs"])),
                    "sampleNo",
                ].iloc[0]

                #Get the first response period (res) textmark after "apo"
                sample_no_res = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx) & (filtered_textmarks["text"] == "res"), "sampleNo"
                ].iloc[0]

                # Get the next "rat" textmark (signalling rating) after anticipation post name
                sample_no_rat = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx)
                    & (filtered_textmarks["text"] == "rat"),
                    "sampleNo",
                ].iloc[0]

                #Convert indices to integers
                sample_no_apr = int(sample_no_apr)
                sample_no_apo = int(sample_no_apo)
                sample_no_shock = int(sample_no_shock)
                sample_no_res = int(sample_no_res)
                sample_no_rat = int(sample_no_rat)

                """ # Important - check durations of each phase are correct
                #Waiting phase
                if (sample_no_apo - sample_no_apr > 3100) | (sample_no_apo - sample_no_apr < 2900) :
                     print(RED + '- Trial ' +  str(trial_no) + ', Waiting Phase is ' + str(sample_no_apo - sample_no_apr) + ' ms' + RESET)
                     #print(BLUE + '- Using 3000 ms as Waiting Phase duration' + RESET)

                #Response phase
                if (sample_no_rat - sample_no_res > 6100) | (sample_no_rat - sample_no_res < 5900):
                    #print(RED + '- Trial ' +  str(trial_no) + ', Response Phase > 6100 ms' + RESET)
                    print(RED + '- Trial ' +  str(trial_no) + ', Response Phase is ' + str(sample_no_rat - sample_no_res) + ' ms' + RESET)

                #Process exceptions
                #> For BG035 trial number 2 'rat' textmark is late so trim to 6000 ms
                if (sessionID == 'BG035') & (trial_no == 19):
                    sample_no_rat = sample_no_res + 6000
                    print(BLUE + '- Trial ' +  str(trial_no) + ', Corrected Response Phase textmarks to 6000 ms' + RESET) """

                #Anticipation phase
                if (sample_no_shock - sample_no_apo > 10100) | (sample_no_shock - sample_no_apo < 5900):
                    #print(RED + '- Trial ' +  str(trial_no) + ', Anticipation Phase is ' + str(sample_no_shock - sample_no_apo) + ' ms' + RESET)
                    
                    #If antiicpation phase is incorrect length, work out shock time as 500ms before start of response phase
                    sample_no_shock = sample_no_res - DURATION_SHOCK
                    #print(BLUE + '- Trial ' +  str(trial_no) + ', New Anticipation Phase duration: ' + str(sample_no_shock - sample_no_apo) + ' ms' + RESET)
                    
                    #if this does not fix it then print error
                    if (sample_no_shock - sample_no_apo > 10100) | (sample_no_shock - sample_no_apo < 5900):
                        print(RED + '- ERROR: Trial ' +  str(trial_no) + ', Anticipation Phase is ' + str(sample_no_shock - sample_no_apo) + ' ms even after using "res" textmark.' + RESET)
                

                #------ Calculate HR measures for each phase

                # Calculate mean HR for each phase
                mean_hr_waiting = calculate_mean_hr(sample_no_apr, sample_no_apr + DURATION_WAITING, subj_inst_hr, subj_noisy_samples, SPIKE_FS)
                mean_hr_anticipation = calculate_mean_hr(sample_no_apo, sample_no_shock, subj_inst_hr, subj_noisy_samples, SPIKE_FS)
                mean_hr_response = calculate_mean_hr(sample_no_shock + DURATION_SHOCK, sample_no_shock + DURATION_SHOCK + DURATION_RESPONSE_EXCL_SHOCK, subj_inst_hr, subj_noisy_samples, SPIKE_FS) #do not include shock

                # Calculate instantaneous HR timeseries during each phase
                hr_timeseries_waiting = calculate_hr_timeseries(sample_no_apr - DURATION_PRE_PHASE, sample_no_apr + DURATION_WAITING, subj_inst_hr, subj_noisy_samples, orig_fs = SPIKE_FS, target_fs = 5)
                hr_timeseries_anticipation = calculate_hr_timeseries(sample_no_apo - DURATION_PRE_PHASE, sample_no_shock, subj_inst_hr, subj_noisy_samples, orig_fs = SPIKE_FS, target_fs = 5)
                hr_timeseries_response = calculate_hr_timeseries(sample_no_shock - DURATION_PRE_PHASE, sample_no_shock + DURATION_SHOCK + DURATION_RESPONSE_EXCL_SHOCK, subj_inst_hr, subj_noisy_samples, orig_fs = SPIKE_FS, target_fs = 5) #include shock in timeseries

                #if subject_row is valid then add HR and HR change values to dataframe
                if sum(subject_row) == 1:
                    # Add mean HR values to the dataframe
                    df_all_combined.loc[subject_row, "mean_hr_waiting"] = round(mean_hr_waiting, 4)
                    df_all_combined.loc[subject_row, "mean_hr_anticipation"] = round(mean_hr_anticipation, 4)
                    df_all_combined.loc[subject_row, "mean_hr_response"] = round(mean_hr_response, 4)

                    # Add HR timeseries to the dataframe
                    #>Assign as a string representation of the list
                    df_all_combined.loc[subject_row, 'hr_series_waiting'] = str(hr_timeseries_waiting.tolist())
                    df_all_combined.loc[subject_row, 'hr_series_anticipation'] = str(hr_timeseries_anticipation.tolist())
                    df_all_combined.loc[subject_row, 'hr_series_response'] = str(hr_timeseries_response.tolist())

                    #Process exceptions

                    #> BG021 trial 79 crashed during response phase so set responses to nan
                    if (sessionID == 'BG021') & (trial_no == 79):
                        df_all_combined.loc[subject_row, "mean_hr_response"] = np.nan
                        df_all_combined.loc[subject_row, 'hr_series_response'] = str( np.array([np.nan]) )
                        print(BLUE + 'Set HR responses for trial 79 resposne phase to nan' + RESET)

                    #> BG040c has very long IBI immediately before trial 40. HOWEVER this is a genuine long IBI so have left unchanged. 

                elif sum(subject_row) > 1 :
                    print(RED + f"WARNING: multiple subject_rows {subject_row} is in df_all_combined." + RESET)
                elif sum(subject_row) == 0 :
                    print(RED + f"WARNING: subject_row {subject_row} is not in df_all_combined." + RESET)

            #Print message
            print(BLUE + '- Successfully calculated HR resposnes' + RESET)

        #Process exceptions (BG047 - exclude all HR data due to ventricular arrhythmia)
        if subjID == 'BG047c': 
            print(BLUE + '- Excluded BG047c ECG data' + RESET)
            # Mean HR
            df_all_combined.loc[subj_rows_combined, "mean_hr_waiting"] = pd.NA
            df_all_combined.loc[subj_rows_combined, "mean_hr_anticipation"] = pd.NA
            df_all_combined.loc[subj_rows_combined, "mean_hr_response"] = pd.NA

            #HR timeseries
            df_all_combined.loc[subj_rows_combined, "hr_series_waiting"] = str( np.array([np.nan]) )
            df_all_combined.loc[subj_rows_combined, "hr_series_anticipation"] = str( np.array([np.nan]) )
            df_all_combined.loc[subj_rows_combined, 'hr_series_response'] = str( np.array([np.nan]) )

            

Session:  BG001
- Importing Spike data for BG001
BG001p peak deletion/addition
- Successfully deleted peaks at [2039311 2188113 2416994 2417234 2582198 3310513 3726899 4009342 4424940
 4464605 4527491 4539044 4619962 4776050 4842699 4922331 4922558 5155788
 5365235 5540069 5669440 5751039 5751247]
- Added missing peaks  [2417320 4922507]
BG001c peak deletion/addition
ERROR: No peaks to delete at sample nos.  [3454816 4128070 4128326] 
However peaks found on previous samples [3454815 4128069 4128325]
- Successfully deleted peaks from previous samples at [3454815 4128069 4128325]
BG001p calculating HR responses
- No. of apr textmarks (80) matches no. rows (80) data for this subject in df_all_combined
- Processing noisy sections
- Successfully calculated HR resposnes
BG001c calculating HR responses
- No. of apr textmarks (80) matches no. rows (80) data for this subject in df_all_combined
- Successfully calculated HR resposnes
Session:  BG002
- Importing Spike data for BG002
BG002p peak de

Save as csv
- Option to save for debugging

In [ ]:
#save_destination = os.path.join(BASE_DIR / "Participant data" / "Preprocessed task data" / "Empathy task",
#                                "df_all_combined_withHR.csv")

#df_all_combined.to_csv(save_destination, index=False)  # Optional: index=False to avoid writing row indices


Import data from participant notes

In [ ]:
from docx import Document

def extract_text_from_docx(doc_path):

    """
    Extract all readable text from a Word (.docx) document.

    Text is collected from both paragraphs and tables. Empty paragraphs and
    empty table cells are ignored. Table rows are returned as a single string
    with cell contents separated by " | ".

    Args:
        doc_path (str or Path): File path to the .docx document.

    Returns:
        str: Extracted document text as a single newline-separated string.

    Notes:
        Requires `Document` from the `python-docx` package.
    """

    # Load the Word document
    document = Document(doc_path)
    
    # Collect all text content
    text_content = []

    # Extract text from paragraphs
    for paragraph in document.paragraphs:
        if paragraph.text.strip():  # Exclude empty paragraphs
            text_content.append(paragraph.text)

    # Extract text from tables
    for table in document.tables:
        for row in table.rows:
            row_text = [cell.text.strip() for cell in row.cells if cell.text.strip()]  # Get non-empty cell text
            if row_text:  # Add row content if not empty
                text_content.append(" | ".join(row_text))  # Combine cell text with ' | ' for readability

    return "\n".join(text_content)

#predefine
df_all_notes = pd.DataFrame()

for sessionID in session_folders:

    # get session folder
    sessionFolder = os.path.join(directory, sessionID)
    print('Session: ', sessionID) 

    print(sessionID + ' extracting word doc')

    # Path to the Word document
    doc_path = sessionFolder + '/' +  sessionID + ' notes.docx'

    # Extract and print the text
    full_text = extract_text_from_docx(doc_path)

    #generate IDs of both subjects in session
    subjsInSession = ["p", "c"]

    for subj in subjsInSession:
        
        subjID = sessionID + subj

        print(subjID + ' extracting blood pressure')
        
        # Convert to lowercase for case-insensitive search
        full_text = full_text.lower()
        
        # Find all occurrences of "companion"
        companion_indices = [i for i in range(len(full_text)) if full_text.startswith("companion", i)]

        # If there is only one occurence then use it to split the text
        if len(companion_indices) > 1:
            print(RED + "Warning: Multiple occurrences of 'companion' found." + RESET)
        elif len(companion_indices) == 1:
            if subj == "p":
                #take text up to word 'companion'
                subj_text = full_text[:companion_indices[0]].strip()
            elif subj == "c":
                #take text from word 'companion' to end
                subj_text = full_text[companion_indices[0]:].strip()
        else:
            print(RED + "Warning: No occurrences of 'companion' found." + RESET)


        #extract arm blood pressure
        bp_string = "blood pressure:"
        blood_pressure_indices = [i for i in range(len(subj_text)) if subj_text.lower().startswith(bp_string, i)]
        bp_text = subj_text[blood_pressure_indices[-1] + + len(bp_string): blood_pressure_indices[-1] + len(bp_string) +7].strip()
        print('- Blood pressure = '  + bp_text)
        bp_sys, bp_dia = map(int, bp_text.split('/')) #Split blood pressure text

        #extract EDA range
        eda_string = "eda range:"
        eda_range_pressure_indices = [i for i in range(len(subj_text)) if subj_text.lower().startswith(eda_string, i)]
        eda_range = int(subj_text[eda_range_pressure_indices[-1] + len(eda_string): eda_range_pressure_indices[-1] + len(eda_string) +4].strip())
        print('- EDA range = '  + str(eda_range))

        # Extract text between 'hct' and 'hct psychophys'
        hct_start_string = "hct"
        hct_end_string = "hct psychophys"

        #Extract notes about hct performance
        hct_start_index = subj_text.lower().find(hct_start_string)
        hct_end_index = subj_text.lower().find(hct_end_string)

        if hct_start_index != -1 and hct_end_index != -1 and hct_start_index < hct_end_index:
            hct_text = subj_text[hct_start_index + len(hct_start_string):hct_end_index].strip()
        else:
            print(RED + 'ERROR - problems finding hct_start_index or hct_end_index')
            hct_text = None  # Set to None if 'hct' or 'hct psychophys' is not found in the correct order

        # Add this subject's notes data to df_all_combined
        df_notes = pd.DataFrame()
        df_notes.loc[0,'subjID'] = subjID
        df_notes.loc[0,'all_notes'] = subj_text
        df_all_notes = pd.concat([df_all_notes, df_notes])
        p_rows_combined = df_all_combined['subjectID'].str.contains((subjID), na=False, case=False) #get logical array of rows containing participant ID
        df_all_combined.loc[p_rows_combined, 'all_notes'] = subj_text
        df_all_combined.loc[p_rows_combined, 'arm_sys'] = bp_sys
        df_all_combined.loc[p_rows_combined, 'arm_dia'] = bp_dia
        df_all_combined.loc[p_rows_combined, 'eda_range'] = eda_range

#save df_all_notes
df_all_notes.to_csv('Data checks/df_all_notes.csv')


Session:  BG001
BG001 extracting word doc
BG001p extracting blood pressure
- Blood pressure = 120/72
- EDA range = 25
BG001c extracting blood pressure
- Blood pressure = 123/74
- EDA range = 25
Session:  BG002
BG002 extracting word doc
BG002p extracting blood pressure
- Blood pressure = 109/71
- EDA range = 25
BG002c extracting blood pressure
- Blood pressure = 115/63
- EDA range = 25
Session:  BG003
BG003 extracting word doc
BG003p extracting blood pressure
- Blood pressure = 109/70
- EDA range = 25
BG003c extracting blood pressure
- Blood pressure = 124/73
- EDA range = 10
Session:  BG004
BG004 extracting word doc
BG004p extracting blood pressure
- Blood pressure = 106/72
- EDA range = 25
BG004c extracting blood pressure
- Blood pressure = 122/74
- EDA range = 25
Session:  BG005
BG005 extracting word doc
BG005p extracting blood pressure
- Blood pressure = 115/70
- EDA range = 25
BG005c extracting blood pressure
- Blood pressure = 128/80
- EDA range = 25
Session:  BG006
BG006 extracti

Function for plotting EDA data

In [ ]:
def plot_eda(sessionID, p_or_c, eda_data, df_textmarks, transfer_constant, save_fig):
  
    """
    Plot EDA (electrodermal activity) signal for a session with key task textmarks.

    The function applies a transfer constant to convert raw values, detrends the signal,
    extracts the phasic component using NeuroKit2, downsamples for visualization, and
    plots raw, detrended, and phasic EDA traces using Plotly. Task textmarks are added
    as annotated vertical lines.

    Args:
        sessionID (str): Session identifier.
        p_or_c (str): "p" for participant or "c" for companion.
        eda_data: Neo analog signal object containing EDA data and `.times`.
        df_textmarks (pd.DataFrame): DataFrame of textmarks with 'text' and 'sampleNo' columns.
        transfer_constant (float): Scaling factor applied to the raw EDA signal.
        save_fig (bool): If True, saves the figure as a PNG.

    Returns:
        None: Displays an interactive Plotly figure and optionally saves it.

    Notes:
        - Assumes `SPIKE_FS` is defined globally for converting sample numbers to seconds.
        - Requires Plotly (`plotly.graph_objs as go`) and NeuroKit2 (`neurokit2 as nk`).
        - Downsamples by a fixed factor for plotting only.
        - Saving requires `BASE_DIR` to be defined globally.
    """

    #get details of eda data
    eda_signal = eda_data.magnitude.squeeze()

    #apply transfer constant
    eda_signal = eda_signal * transfer_constant

    # Detrending removes linear drift (fits a linear regression line to the data and subtracts it from the signal)
    detrended_signal = detrend(eda_signal)

    #calculate SCRs
    eda_measures = nk.eda_phasic(np.array(eda_signal), sampling_rate=1000)
    eda_phasic = np.array(eda_measures['EDA_Phasic'])

    # Convert Neo times to numerical array
    times = eda_data.times.magnitude.squeeze()

    # Downsample for plot
    downsample_factor = 100 # Keep every Nth sample
    eda_signal = eda_signal[::downsample_factor]
    detrended_signal = detrended_signal[::downsample_factor]
    eda_phasic = eda_phasic[::downsample_factor]
    times = times[::downsample_factor]

    # Add the time series data
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=times, y=eda_signal, mode='lines', name='Raw')) #plot eda trace
    fig.add_trace(go.Scatter(x=times, y=detrended_signal, mode='lines', name='Detrended')) #plot detrended data
    fig.add_trace(go.Scatter(x=times, y=eda_phasic, mode='lines', name='Phasic')) #plot detrended data

    ## Add textmarks
    textmarks_of_interest = ['apr', 'rat', 'sph', 'spl', 'sch', 'scl', 'sps', 'scs'] #apr = anticipation before name displayed; rat = subjects prompted to submit rating
    df_textmarks_subset = df_textmarks[df_textmarks['text'].isin(textmarks_of_interest)]
    for _, row in df_textmarks_subset.iterrows():
        tm_sample_no = row['sampleNo']
        tm_text = row['text']
        
        # Add vertical line
        fig.add_vline(x=tm_sample_no/(SPIKE_FS), line_width=2, line_dash="dash", line_color="grey") #divide sample number by SPIKE_FS to convert to s
        
        # Add text annotation
        fig.add_annotation(
            x=tm_sample_no/(SPIKE_FS),
            y=0,  # Adjust this value to position the text vertically
            text=tm_text,
            showarrow=False,
            textangle=-90,
            yanchor="bottom",
            font=dict(color="black", size=20)
        )

    if p_or_c == 'p':
        part_type = 'participant'
    else:
        part_type = 'companion'

    fig.update_layout(
        title = f"{sessionID}: {part_type} EDA data"
    )
    
    fig.show()

    #save
    if save_fig:
        saveDestination = = BASE_DIR / "Analysis" / "Data checks" / ""
        fig.update_layout( #update dimensions before saving
        autosize=False,
        width=1000,
        height=250)

        fig.write_image(f"{saveDestination}{sessionID}_{part_type}_eda.png")

        print("saved figure as .png on OneDrive")

Check EDA data (figures)

In [ ]:
plot_eda_data = False #define whether to run this section or skip it

if plot_eda_data:
    import warnings # Suppress the specific DeprecationWarning

    warnings.filterwarnings("ignore", category=DeprecationWarning)

    # Visual verification variables
    eda_visual_verification = True
    save_eda_figs = False
    num_plotted = 0
    max_plotted = 100

    #------- Import excel file containing notes from ecg inspection
    analysisFolder = BASE_DIR / "Analysis"
    eda_inspection_file = os.path.join(analysisFolder, 'Data checks', 'empathy_eda_inspection.xlsx')
    df_eda_inspection = pd.read_excel(eda_inspection_file)

    for sessionID in session_folders:

        # get session folder
        sessionFolder = os.path.join(directory, sessionID)
        print('Session: ', sessionID) 
        
        # Import Spike2 data file
        spike_file_extension = "empathy.smrx"
        spike_search_pattern = os.path.join(sessionFolder, f'*{spike_file_extension}')
        spike_files = glob.glob(spike_search_pattern)
        if len(spike_files) > 1:
            print("-", "{} SMRX files found.".format(len(spike_files)))
        elif len(spike_files) < 1:
            print(RED + 'WARNING: no empathy spike data files found')

        # Process exceptions
        if sessionID == 'BG020':
            #For some reason, BG020 spike file is not being imported correctly so have duplicated, which works fine
            spike_files = BASE_DIR / "Participant data" / "BG020" / "BG020_empathy_duplicate.smrx"

        # Loop over files
        for i_spike_file in range(len(spike_files)):
            
            spike_filename = spike_files[i_spike_file]

            print("-", "Importing Spike data for {}".format(sessionID))

            # Read Spike2 file
            reader = CedIO(filename=spike_filename)
            
            block = reader.read_block()
            spike2data = block.segments[0] # single segment

            # Create a dictionary of analog channels and their indexes
            analog_channel_dictionary = {}

            for data_channel in range(len(spike2data.analogsignals)):
                channel_idx = data_channel
                channel_name = spike2data.analogsignals[data_channel].array_annotations['channel_names'][0]
                #print(channel_name)
                analog_channel_dictionary[channel_name] = channel_idx

            # ---------- Get the EDA data for the subjects
            p_eda_data = spike2data.analogsignals[analog_channel_dictionary['p_EDA']] #*IMPORTANT
            c_eda_data = spike2data.analogsignals[analog_channel_dictionary['c_EDA']] #*IMPORTANT

        
        # ---------- Extract textmarks from smrx file using SonPy
        smrfile = sonpy.lib.SonFile(spike_filename, True)

        textmarkChanNo = 30
        chanIdxInFile = textmarkChanNo - 1

        max_n_tick = smrfile.ChannelMaxTime(chanIdxInFile)

        n_chunks = 4
        breakpoints = divide_into_n_breakpoints(max_n_tick, n_chunks)
        df_textmarks_allchunks = pd.DataFrame()

        for i_chunk in range(n_chunks):
            first_sample = breakpoints[i_chunk]
            last_sample = breakpoints[i_chunk + 1]

            # Adjust the last chunk to ensure it includes the final tick
            if i_chunk == n_chunks - 1:
                last_sample += 1  # Add 1 to include the very last tick
            
            nSamplesToImport = last_sample - first_sample
            textmarks = smrfile.ReadTextMarks(chan=chanIdxInFile, nMax=nSamplesToImport, tFrom=first_sample, tUpto=last_sample)

            textmarks_sampleNo = np.full(len(textmarks), np.nan)
            textmarks_text = np.array(['---'] * len(textmarks), dtype=str)

            for i_mark in range(len(textmarks)):
                mark = textmarks[i_mark]
                textmarks_sampleNo[i_mark] = round(mark.Tick / 100)
                textmarks_text[i_mark] = ''.join([str(mark[0]), str(mark[1]), str(mark[2])])
            
            df_textmarks = pd.DataFrame({'sampleNo': textmarks_sampleNo, 'text': textmarks_text})
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, df_textmarks], ignore_index=True)

        #IMPORTANT - process textmark exceptions
        if sessionID == 'BG002':
            #start from row idx 19
            df_textmarks_allchunks = df_textmarks_allchunks.iloc[19:]

        elif sessionID == 'BG021':
            #add rat textmark at last sample of spike file (when it crashed)
            new_row_rating = pd.DataFrame({"sampleNo": [5509836], "text": ['rat']})  # Create a new DataFrame with the new row
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, new_row_rating], ignore_index=True)  # Append the row  

        #IMPORTANT - calculate scaling factors based off input range    
        dataRange = 10 #CED 2502 data is acquired between -5 and +5

        #calculate participant scaling factor
        p_id = sessionID + 'p'
        row1_p_combined = df_all_combined.index[df_all_combined["subjectID"].str.lower() == p_id.lower()][0]
        p_eda_range = df_all_combined.loc[row1_p_combined, 'eda_range'] #get input range
        if isinstance(p_eda_range, pd.Series):
            # If 'p_eda_range' is still a series with multiple rows, retrieve the first element
            p_eda_range = p_eda_range.iloc[0]  # Get the first value
        
        p_transferConstant = p_eda_range/dataRange #calculate participant transfer constant
        print('- p transfer constant = ' + str(p_transferConstant))

        #calculate companion scaling factor
        c_id = sessionID + 'c'
        row1_c_combined = df_all_combined.index[df_all_combined["subjectID"].str.lower() == c_id.lower()][0]
        c_eda_range = df_all_combined.loc[row1_c_combined, 'eda_range'] #get input range
        if isinstance(c_eda_range, pd.Series):
            # If 'c_eda_range' is still a series with multiple rows, retrieve the first element
            c_eda_range = c_eda_range.iloc[0]  # Get the first value
        c_transferConstant = c_eda_range/dataRange #calculate companion transfer constant
        print('- c transfer constant = ' + str(c_transferConstant))

        #IMPORTANT - plot eda data
        plot_eda(sessionID, 'p', p_eda_data, df_textmarks_allchunks, p_transferConstant, save_eda_figs)
        plot_eda(sessionID, 'c', c_eda_data, df_textmarks_allchunks, c_transferConstant, save_eda_figs)

Calculate EDA measures

https://neuropsychology.github.io/NeuroKit/functions/eda.html#eda-process
nk.eda_process cleans data using neurokit defaults (Low-pass filter with a 3 Hz cutoff frequency and a 4th order Butterworth filter), 
then identifies SCRs using a high pass filter with a cutoff frequency of 0.05 Hz,
calculates tonic component by subtracting SCRs from signal
N.B. the way I am doing it below is better than eda_eventrelated because eda_eventrelated only counts the first SCR in the epoch.
Using neurokit deafults, SCRs are excluded if their amplitude is less than 10% of the largest SCR amplitude in the timeseries (amplitude_min=0.1)

In [ ]:
run_eda_analyses = True #define whether to run this section or skip it
standardise_eda_range_within_subjs = False #define whether to standardise (z score) EDA values within subjects

if run_eda_analyses:
    import warnings # Suppress the specific DeprecationWarning

    warnings.filterwarnings("ignore", category=DeprecationWarning)

    #------- Import excel file containing notes from ecg inspection
    analysisFolder = BASE_DIR / "Analysis"
    eda_inspection_file = os.path.join(analysisFolder, 'Data checks', 'empathy_eda_inspection.xlsx')
    df_eda_inspection = pd.read_excel(eda_inspection_file)

    for sessionID in session_folders:

        # get session folder
        sessionFolder = os.path.join(directory, sessionID)
        print('Session: ', sessionID) 
        
        # Import Spike2 data file
        spike_file_extension = "empathy.smrx"
        spike_search_pattern = os.path.join(sessionFolder, f'*{spike_file_extension}')
        spike_files = glob.glob(spike_search_pattern)
        if len(spike_files) > 1:
            print("-", "{} SMRX files found.".format(len(spike_files)))
        elif len(spike_files) < 1:
            print(RED + 'WARNING: no empathy spike data files found')

        # Process exceptions
        if sessionID == 'BG020':
            #For some reason, BG020 spike file is not being imported correctly so have duplicated, which works fine
            spike_files = BASE_DIR / "Participant data" / "BG020" / "BG020_empathy_duplicate.smrx"

        # Loop over files
        for i_spike_file in range(len(spike_files)):
            
            spike_filename = spike_files[i_spike_file]

            print("-", "Importing Spike data for {}".format(sessionID))

            # Read Spike2 file
            reader = CedIO(filename=spike_filename)
            
            block = reader.read_block()
            spike2data = block.segments[0] # single segment

            # Create a dictionary of analog channels and their indexes
            analog_channel_dictionary = {}

            for data_channel in range(len(spike2data.analogsignals)):
                channel_idx = data_channel
                channel_name = spike2data.analogsignals[data_channel].array_annotations['channel_names'][0]
                #print(channel_name)
                analog_channel_dictionary[channel_name] = channel_idx

            # ---------- Get the EDA data for the subjects
            p_eda_data = spike2data.analogsignals[analog_channel_dictionary['p_EDA']] #*IMPORTANT
            c_eda_data = spike2data.analogsignals[analog_channel_dictionary['c_EDA']] #*IMPORTANT

        
        # ---------- Extract textmarks from smrx file using SonPy
        smrfile = sonpy.lib.SonFile(spike_filename, True)

        textmarkChanNo = 30
        chanIdxInFile = textmarkChanNo - 1

        max_n_tick = smrfile.ChannelMaxTime(chanIdxInFile)

        n_chunks = 4
        breakpoints = divide_into_n_breakpoints(max_n_tick, n_chunks)
        df_textmarks_allchunks = pd.DataFrame()

        for i_chunk in range(n_chunks):
            first_sample = breakpoints[i_chunk]
            last_sample = breakpoints[i_chunk + 1]

            # Adjust the last chunk to ensure it includes the final tick
            if i_chunk == n_chunks - 1:
                last_sample += 1  # Add 1 to include the very last tick
            
            nSamplesToImport = last_sample - first_sample
            textmarks = smrfile.ReadTextMarks(chan=chanIdxInFile, nMax=nSamplesToImport, tFrom=first_sample, tUpto=last_sample)

            textmarks_sampleNo = np.full(len(textmarks), np.nan)
            textmarks_text = np.array(['---'] * len(textmarks), dtype=str)

            for i_mark in range(len(textmarks)):
                mark = textmarks[i_mark]
                textmarks_sampleNo[i_mark] = round(mark.Tick / 100)
                textmarks_text[i_mark] = ''.join([str(mark[0]), str(mark[1]), str(mark[2])])
            
            df_textmarks = pd.DataFrame({'sampleNo': textmarks_sampleNo, 'text': textmarks_text})
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, df_textmarks], ignore_index=True)

        #IMPORTANT - process textmark exceptions
        if sessionID == 'BG002':
            #start from row idx 19
            df_textmarks_allchunks = df_textmarks_allchunks.iloc[19:]

        elif sessionID == 'BG021':
            #add rat textmark at last sample of spike file (when it crashed)
            new_row_rating = pd.DataFrame({"sampleNo": [5509836], "text": ['rat']})  # Create a new DataFrame with the new row
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, new_row_rating], ignore_index=True)  # Append the row  

        #IMPORTANT - calculate scaling factors based off input range    
        dataRange = 10 #CED 2502 data is acquired between -5 and +5

        #calculate participant scaling factor
        p_id = sessionID + 'p'
        row1_p_combined = df_all_combined.index[df_all_combined["subjectID"].str.lower() == p_id.lower()][0]
        p_eda_range = df_all_combined.loc[row1_p_combined, 'eda_range'] #get input range
        if isinstance(p_eda_range, pd.Series):
            # If 'p_eda_range' is still a series with multiple rows, retrieve the first element
            p_eda_range = p_eda_range.iloc[0]  # Get the first value
        p_transferConstant = p_eda_range/dataRange #calculate participant transfer constant
        print('- p transfer constant = ' + str(p_transferConstant))

        #calculate companion scaling factor
        c_id = sessionID + 'c'
        row1_c_combined = df_all_combined.index[df_all_combined["subjectID"].str.lower() == c_id.lower()][0]
        c_eda_range = df_all_combined.loc[row1_c_combined, 'eda_range'] #get input range
        if isinstance(c_eda_range, pd.Series):
            # If 'c_eda_range' is still a series with multiple rows, retrieve the first element
            c_eda_range = c_eda_range.iloc[0]  # Get the first value
        c_transferConstant = c_eda_range/dataRange #calculate companion transfer constant
        print('- c transfer constant = ' + str(c_transferConstant))
    
        #----- Downsample data
        downsample_factor = 1 #factor to downsample by (N.B. factor not Hz)

        #--- Downsample textmarks for calculating EDA responses
        df_textmarks_allchunks['sampleNo'] = (df_textmarks_allchunks['sampleNo'] / downsample_factor).round(0).astype(int) #downsample textmarks
        
        #generate IDs of both subjects in session
        subjsInSession = ["p", "c"]

        for subj in subjsInSession:
            
            subjID = sessionID + subj

            if subj == "p": #IMPORTANT - defines array for calculating EDA responses
                subj_eda_data = p_eda_data
                subj_transfer_constant = p_transferConstant
            elif subj == "c":
                subj_eda_data = c_eda_data
                subj_transfer_constant = c_transferConstant
            else:
                print(RED + 'ERROR - issue defining subj_eda_data - subj not recognised')

            #Process exceptions
            if subjID == 'BG006p':
                #BG006 p and c psychophys channels are wrong way round for empathy task. Swap over!
                subj_eda_data = c_eda_data
            elif subjID == 'BG006c':
                 subj_eda_data = p_eda_data

            #---- Calculate mean SCL, SCR count, sum SCR amplitudes for each phase
            print(subjID + ' Processing EDA data')

            #---- create array specifying noisy sections of EDA data
            row_subj_recording = (df_eda_inspection['Subject'] == subjID)# Find rows in ecg inspection excel file for this subject
            eda_idx_noisy_section = df_eda_inspection.loc[row_subj_recording, 'EDA_bad_segments'] # Get sample numbers of entire sections of peaks to delete

            if isinstance(eda_idx_noisy_section, str): # Ensure it's a list or iterable of strings, in case it's a single value
                eda_idx_noisy_section = [eda_idx_noisy_section]

            eda_idx_noisy_samples = [] # Initialise an empty list to collect all indices to delete

            for cell_value in eda_idx_noisy_section: # Process each cell value
                if cell_value.lower() == 'none':  # Skip cells with 'none'
                    continue
                for section in cell_value.split(','):  # Split multiple ranges by commas
                    print(BLUE + '- Processing noisy sections' + RESET)
                    start, end = map(int, section.split(':'))  # Split each range and convert to integers
                    eda_idx_noisy_samples.extend(range(start, end + 1))  # Include the endpoint
            
            eda_noisy_samples = np.full(len(subj_eda_data), False, dtype=bool) #define logical array
            if len(eda_idx_noisy_samples) > 0: #downsample any sample numbers
                eda_idx_noisy_samples = [round(int(idx)/downsample_factor) for idx in eda_idx_noisy_samples]
                eda_noisy_samples[eda_idx_noisy_samples] = True #set noisy trials to true
            
            print('- Calculated noisy samples')

            #----- Transform EDA signal (convert to microsiemens, filter low freq. drift ) -updated 15.04.25
            subj_eda_signal = subj_eda_data.magnitude.squeeze() #Get eda data
            subj_eda_signal = subj_eda_signal + 5 #add 5 to make all values positive
            subj_eda_signal = subj_eda_signal * subj_transfer_constant #apply transfer constant to convert to microsiemens
            print('- Applied transfer constant ' + str(round(subj_transfer_constant, 2)))
            
            
            if standardise_eda_range_within_subjs:
                subj_eda_signals_nk, subj_scr_info_nk = nk.eda_process(nk.standardize(subj_eda_signal), sampling_rate = SPIKE_FS) #standardised (z scored within subjects)
                print(BLUE + '- Calculated STANDARDISED tonic + phasic components for entire timeseries' + RESET)
            else:
                subj_eda_signals_nk, subj_scr_info_nk = nk.eda_process(subj_eda_signal, sampling_rate = SPIKE_FS) #unstandardised
                print('- Calculated unstandardised tonic + phasic components for entire timeseries')

            #-------- Identify textmarks for each trial
            #check that number of anticipation and shock textmarks matches number of trials
            nTrialsInTextmarks = len(df_textmarks_allchunks[df_textmarks_allchunks['text'].isin(['apr'])]) #get number of trials in textmarks
            subj_rows_combined = df_all_combined['subjectID'].str.contains(subjID, na=False, case=False)  #get subject rows in df_all_combined

            if nTrialsInTextmarks != sum(subj_rows_combined):
                print(RED + 'ERROR: there are ' + str(nTrialsInTextmarks) + 
                    ' apr textmarks but ' + str(sum(subj_rows_combined)) + 
                    ' apr textmarks but ' + str(sum(subj_rows_combined)) + 
                    ' rows of data for this subject in df_all_combined' + RESET)
            else:
                print('- No. of apr textmarks (' + str(nTrialsInTextmarks) + 
                    ') matches no. rows (' + str(sum(subj_rows_combined)) + 
                    ') data for this subject in df_all_combined')


            # Iterate through each trial and calculate SCL/SCR measures for each phase
            #> Filter rows for the relevant textmarks in df_textmarks_all_chunks
            relevant_textmarks = ["apr", #anticipation pre-name
                                  "apo", #anticipation post-name
                                  "sph", "spl", "sps", "sch", "scl", "scs", #shock delivery
                                  "res", #start of response period
                                  "rat"] #submit rating/experimenter change shock settings
            filtered_textmarks = df_textmarks_allchunks[df_textmarks_allchunks["text"].isin(relevant_textmarks)]

            # Find all indices of the "apr" textmark (one for each trial)
            apr_indices = filtered_textmarks[filtered_textmarks["text"] == "apr"].index

            # Iterate through each trial using "apr" textmarks as the starting point
            for trial_idx, apr_idx in enumerate(apr_indices):
                
                #Calculate trial number (start at 1 not 0)
                trial_no = trial_idx + 1
                
                #calculate subject row as logical array
                subject_row = (df_all_combined['subjectID'].str.contains(subjID, na=False, case=False)) & (df_all_combined['trialNo'] == trial_no)

                # Get the indices of the textmarks for this trial
                sample_no_apr = int(filtered_textmarks.loc[apr_idx, "sampleNo"])

                # Get the next "apo" textmark (signalling start of anticipation post name period) after "apr"
                sample_no_apo = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx) & (filtered_textmarks["text"] == "apo"), "sampleNo"
                ].iloc[0]

                # Get the first shock (sph, spl, sps, sch, scl, scs) textmark after "apo"
                sample_no_shock = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx)
                    & (filtered_textmarks["text"].isin(["sph", "spl", "sps", "sch", "scl", "scs"])),
                    "sampleNo",
                ].iloc[0]

                #Get the first response period (res) textmark after "apo"
                sample_no_res = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx) & (filtered_textmarks["text"] == "res"), "sampleNo"
                ].iloc[0]

                # Get the next "rat" textmark (signalling rating) after anticipation post name
                sample_no_rat = filtered_textmarks.loc[
                    (filtered_textmarks.index > apr_idx)
                    & (filtered_textmarks["text"] == "rat"),
                    "sampleNo",
                ].iloc[0]

                #Convert indices to integers
                sample_no_apr = int(sample_no_apr)
                sample_no_apo = int(sample_no_apo)
                sample_no_shock = int(sample_no_shock)
                sample_no_res = int(sample_no_res)
                sample_no_rat = int(sample_no_rat)

                #Check anticipation phase textmarks
                if (sample_no_shock - sample_no_apo > 10100) | (sample_no_shock - sample_no_apo < 5900):
                    #print(RED + '- Trial ' +  str(trial_no) + ', Anticipation Phase is ' + str(sample_no_shock - sample_no_apo) + ' ms' + RESET)
                    
                    #If antiicpation phase is incorrect length, work out shock time as 500ms before start of response phase
                    sample_no_shock = sample_no_res - DURATION_SHOCK
                    #print(BLUE + '- Trial ' +  str(trial_no) + ', New Anticipation Phase duration: ' + str(sample_no_shock - sample_no_apo) + ' ms' + RESET)
                    
                    #if this does not fix it then print error
                    if (sample_no_shock - sample_no_apo > 10100) | (sample_no_shock - sample_no_apo < 5900):
                        print(RED + '- ERROR: Trial ' +  str(trial_no) + ', Anticipation Phase is ' + str(sample_no_shock - sample_no_apo) + ' ms even after using "res" textmark.' + RESET)

                #-------- Calculate SCL and SCR measures for each phase

                #Waiting
                if any(eda_noisy_samples[sample_no_apr : sample_no_apr + DURATION_WAITING]): #if eda data is noisy then set values to NaN
                        mean_scl_waiting = np.nan # Average SCL
                        n_scr_waiting = np.nan  # SCR count
                        scr_amplitude_sum_waiting = np.nan #sum SCR amplitudes
                else:
                    subj_signals_waiting = subj_eda_signals_nk.iloc[sample_no_apr : sample_no_apr + DURATION_WAITING]
                    mean_scl_waiting = subj_signals_waiting["EDA_Tonic"].mean() # Average SCL
                    n_scr_waiting = subj_signals_waiting["SCR_Peaks"].sum()  # SCR count
                    scr_amplitude_sum_waiting = subj_signals_waiting.loc[subj_signals_waiting["SCR_Peaks"] == 1, "SCR_Amplitude"].sum() #sum SCR amplitudes

                #Anticipation
                if any(eda_noisy_samples[sample_no_apo : sample_no_shock]): #if eda data is noisy then set values to NaN
                        mean_scl_anticipation = np.nan # Average SCL
                        n_scr_anticipation = np.nan  # SCR count
                        scr_amplitude_sum_anticipation = np.nan #sum SCR amplitudes
                else:
                    subj_signals_anticipation = subj_eda_signals_nk.iloc[sample_no_apo : sample_no_shock]
                    mean_scl_anticipation = subj_signals_anticipation["EDA_Tonic"].mean() # Average SCL
                    n_scr_anticipation = subj_signals_anticipation["SCR_Peaks"].sum()  # SCR count
                    scr_amplitude_sum_anticipation = subj_signals_anticipation.loc[subj_signals_anticipation["SCR_Peaks"] == 1, "SCR_Amplitude"].sum() #sum SCR amplitudes

                #Response
                if any(eda_noisy_samples[sample_no_shock + DURATION_SHOCK : sample_no_shock + DURATION_SHOCK + DURATION_RESPONSE_EXCL_SHOCK]): #if eda data is noisy then set values to NaN
                        mean_scl_response = np.nan # Average SCL
                        n_scr_response = np.nan  # SCR count
                        scr_amplitude_sum_response = np.nan #sum SCR amplitudes
                else:
                    subj_signals_response = subj_eda_signals_nk.iloc[sample_no_shock + DURATION_SHOCK : sample_no_shock + DURATION_SHOCK + DURATION_RESPONSE_EXCL_SHOCK]
                    #do not include shock in mean
                    mean_scl_response = subj_signals_response["EDA_Tonic"].mean() # Average SCL
                    n_scr_response = subj_signals_response["SCR_Peaks"].sum()  # SCR count
                    scr_amplitude_sum_response = subj_signals_response.loc[subj_signals_response["SCR_Peaks"] == 1, "SCR_Amplitude"].sum() #sum SCR amplitudes
                    
                #------------ Calculate SCL timeseries for each phase
                target_fs_ts = 5
                
                #Waiting
                if any(eda_noisy_samples[sample_no_apr - DURATION_PRE_PHASE : sample_no_apr + DURATION_WAITING]): #if eda data is noisy then set values to NaN
                    downsampled_scl_waiting_ts = np.array([np.nan])
                else:
                    subj_signals_waiting_ts = subj_eda_signals_nk.iloc[sample_no_apr - DURATION_PRE_PHASE : sample_no_apr + DURATION_WAITING]
                    scl_waiting_ts = subj_signals_waiting_ts["EDA_Tonic"].to_numpy(dtype=float) # SCL
                    downsampled_scl_waiting_ts = np.round(downsample_with_last(scl_waiting_ts, SPIKE_FS, target_fs = target_fs_ts), 2) #downsample

                #Anticipation
                if any(eda_noisy_samples[sample_no_apo - DURATION_PRE_PHASE : sample_no_shock]): #if eda data is noisy then set values to NaN
                    downsampled_scl_anticipation_ts = np.array([np.nan]) 
                else:
                    subj_signals_anticipation_ts = subj_eda_signals_nk.iloc[sample_no_apo - DURATION_PRE_PHASE : sample_no_shock]
                    scl_anticipation_ts = subj_signals_anticipation_ts["EDA_Tonic"].to_numpy(dtype=float) # SCL
                    downsampled_scl_anticipation_ts = np.round(downsample_with_last(scl_anticipation_ts, SPIKE_FS, target_fs = target_fs_ts), 2) #downsample

                #Response
                if any(eda_noisy_samples[sample_no_shock - DURATION_PRE_PHASE : sample_no_shock + DURATION_SHOCK + DURATION_RESPONSE_EXCL_SHOCK]): #if eda data is noisy then set values to NaN
                    downsampled_scl_response_ts = np.array([np.nan]) 
                else:
                    #include shock in timeseries
                    subj_signals_response_ts = subj_eda_signals_nk.iloc[sample_no_shock - DURATION_PRE_PHASE : sample_no_shock + DURATION_SHOCK + DURATION_RESPONSE_EXCL_SHOCK]
                    scl_response_ts = subj_signals_response_ts["EDA_Tonic"].to_numpy(dtype=float) # SCL
                    downsampled_scl_response_ts = np.round(downsample_with_last(scl_response_ts, SPIKE_FS, target_fs = target_fs_ts), 2) #downsample

                #if subject_row is valid then add SCL and SCR values to dataframe
                if sum(subject_row) == 1:
                    # Add mean scl to dataframe
                    df_all_combined.loc[subject_row, "mean_scl_waiting"] = round(mean_scl_waiting, 4)
                    df_all_combined.loc[subject_row, "mean_scl_anticipation"] = round(mean_scl_anticipation, 4)
                    df_all_combined.loc[subject_row, "mean_scl_response"] = round(mean_scl_response, 4)

                    # Add n SCRs to dataframe
                    df_all_combined.loc[subject_row, "n_scr_waiting"] = round(n_scr_waiting, 4)
                    df_all_combined.loc[subject_row, "n_scr_anticipation"] = round(n_scr_anticipation, 4)
                    df_all_combined.loc[subject_row, "n_scr_response"] = round(n_scr_response, 4)

                    # Add sum SCR amplitudes to dataframe
                    df_all_combined.loc[subject_row, "scr_amplitude_sum_waiting"] = round(scr_amplitude_sum_waiting, 4)
                    df_all_combined.loc[subject_row, "scr_amplitude_sum_anticipation"] = round(scr_amplitude_sum_anticipation, 4)
                    df_all_combined.loc[subject_row, "scr_amplitude_sum_response"] = round(scr_amplitude_sum_response, 4)

                    # Add SCL timeseries to the dataframe
                    #>Assign as a string representation of the list
                    df_all_combined.loc[subject_row, 'scl_series_waiting'] = str(downsampled_scl_waiting_ts.tolist())
                    df_all_combined.loc[subject_row, 'scl_series_anticipation'] = str(downsampled_scl_anticipation_ts.tolist())
                    df_all_combined.loc[subject_row, 'scl_series_response'] = str(downsampled_scl_response_ts.tolist())

                    #Process exceptions

                    #> BG021 trial 79 crashed during response phase so set responses to nan
                    if (sessionID == 'BG021') & (trial_no == 79):
                        df_all_combined.loc[subject_row, "mean_scl_response"] = np.nan
                        df_all_combined.loc[subject_row, "n_scr_response"] = np.nan
                        df_all_combined.loc[subject_row, "scr_amplitude_sum_response"] = np.nan
                        df_all_combined.loc[subject_row, 'scl_series_response'] = str( np.array([np.nan]) )
                        print(BLUE + 'Set SCL/SCR responses for trial 79 resposne phase to nan' + RESET)

                elif sum(subject_row) > 1 :
                    print(RED + f"WARNING: multiple subject_rows {subject_row} is in df_all_combined." + RESET)
                elif sum(subject_row) == 0 :
                    print(RED + f"WARNING: subject_row {subject_row} is not in df_all_combined." + RESET)
            
            #Process exceptions
            #> BG007c EDA data is unusable so do not include any EDA data for any trials
            if subjID == 'BG007c':
                df_all_combined.loc[subj_rows_combined, ["mean_scl_waiting", "mean_scl_anticipation", "mean_scl_response"]] = np.nan
                df_all_combined.loc[subj_rows_combined, ["n_scr_waiting", "n_scr_anticipation", "n_scr_response"]] = np.nan
                df_all_combined.loc[subj_rows_combined, ["scr_amplitude_sum_waiting", "scr_amplitude_sum_anticipation", "scr_amplitude_sum_response"]] = np.nan
                df_all_combined.loc[subj_rows_combined, ['scl_series_waiting', 'scl_series_anticipation', 'scl_series_response']] = str( np.array([np.nan]) )
                print(BLUE + '- BG007c set SCL/SCR responses for all trials nan due to unusable EDA data' + RESET)

            #Print message
            print('- Successfully calculated SCL/SCR resposnes')                

Session:  BG001
- Importing Spike data for BG001
- p transfer constant = 2.5
- c transfer constant = 2.5
BG001p Processing EDA data
- Calculated noisy samples
- Applied transfer constant 2.5
- Calculated unstandardised tonic + phasic components for entire timeseries
- No. of apr textmarks (80) matches no. rows (80) data for this subject in df_all_combined
- Successfully calculated SCL/SCR resposnes
BG001c Processing EDA data
- Calculated noisy samples
- Applied transfer constant 2.5
- Calculated unstandardised tonic + phasic components for entire timeseries
- No. of apr textmarks (80) matches no. rows (80) data for this subject in df_all_combined
- Successfully calculated SCL/SCR resposnes
Session:  BG002
- Importing Spike data for BG002
- p transfer constant = 2.5
- c transfer constant = 2.5
BG002p Processing EDA data
- Calculated noisy samples
- Applied transfer constant 2.5
- Calculated unstandardised tonic + phasic components for entire timeseries
- No. of apr textmarks (40) matche

Calculate B2B responses and timeseries

In [ ]:
run_b2b_analyses = True #define whether to run this section or skip it
plot_b2b_data = False #define whether to plot b2b data
median_smooth_b2b_data = True #define whether to median smooth b2b data
how_process_physiocal = "interpolate" # "interpolate" or "exclude" - define whether to interpolate or exclude data around physiocal

#Function for finding (start, end) indices of contiguous runs of `num` in a 1D signal
def find_runs(signal, num):

    """
    Find contiguous runs of a given value in a 1D signal.

    Identifies start and end indices of consecutive segments where `signal == num`.
    End indices are returned in Python slicing format (end is exclusive).

    Args:
        signal (array-like): 1D input signal.
        num (int or float): Value to search for.

    Returns:
        list[tuple[int, int]]: List of (start_idx, end_idx) pairs for each run.

    Example:
        signal = [0, 1, 1, 1, 0, 1]
        num = 1
        returns [(1, 4), (5, 6)]
    """
    
    signal = np.asarray(signal)
    mask = (signal == num)
    diff = np.diff(mask.astype(int))
    
    starts = np.where(diff == 1)[0] + 1  # Starts of runs
    ends = np.where(diff == -1)[0] + 1   # Ends of runs
    
    # Handle edge cases by inserting `num` at position 0 in starts/ends
    if mask[0]:
        starts = np.insert(starts, 0, num)  # Insert `num` at index 0 of starts
    if mask[-1]:
        ends = np.append(ends, len(signal))
    
    return list(zip(starts, ends))

if run_b2b_analyses:

    warnings.filterwarnings("ignore", category=DeprecationWarning)

    # Visual verification variables
    b2b_visual_verif_done = [] #Add IDs of sessions that have had visual verification of b2b data done

    #------- Import excel file containing notes from ecg inspection
    analysisFolder = BASE_DIR / 'Analysis'
    b2b_inspection_file = os.path.join(analysisFolder, 'Data checks', 'empathy_b2b_inspection.xlsx')
    df_b2b_inspection = pd.read_excel(b2b_inspection_file)

    #Predefine hr timeseries columns as objects to allow insertion of entire timeseries into each cell
    df_all_combined['fisys_series_waiting'] = None 
    df_all_combined = df_all_combined.astype({'fisys_series_waiting': 'object'})
    df_all_combined['fidia_series_waiting'] = None
    df_all_combined = df_all_combined.astype({'fidia_series_waiting': 'object'})

    df_all_combined['fisys_series_anticipation'] = None
    df_all_combined = df_all_combined.astype({'fisys_series_anticipation': 'object'})
    df_all_combined['fidia_series_anticipation'] = None
    df_all_combined = df_all_combined.astype({'fidia_series_anticipation': 'object'})

    df_all_combined['fisys_series_response'] = None
    df_all_combined = df_all_combined.astype({'fisys_series_response': 'object'})
    df_all_combined['fidia_series_response'] = None
    df_all_combined = df_all_combined.astype({'fidia_series_response': 'object'})

    for sessionID in session_folders:

        # get session folder
        sessionFolder = os.path.join(directory, sessionID)
        print('Session: ', sessionID) 
        
        # Import Spike2 data file
        spike_file_extension = "empathy.smrx"
        spike_search_pattern = os.path.join(sessionFolder, f'*{spike_file_extension}')
        spike_files = glob.glob(spike_search_pattern)
        if len(spike_files) > 1:
            print("-", "{} SMRX files found.".format(len(spike_files)))
        elif len(spike_files) < 1:
            print(RED + 'WARNING: no empathy spike data files found')

        # Process exceptions
        if sessionID == 'BG020':
            #For some reason, BG020 spike file is not being imported correctly so have duplicated, which works fine
            spike_files = BASE_DIR / "Participant data" / "BG020" / "BG020_empathy_duplicate.smrx"

        # Loop over files
        for i_spike_file in range(len(spike_files)):
            
            spike_filename = spike_files[i_spike_file]

            print("-", "Importing Spike data for {}".format(sessionID))

            # Read Spike2 file
            reader = CedIO(filename=spike_filename)
            
            block = reader.read_block()
            spike2data = block.segments[0] # single segment

            # Create a dictionary of analog channels and their indexes
            analog_channel_dictionary = {}

            for data_channel in range(len(spike2data.analogsignals)):
                channel_idx = data_channel
                channel_name = spike2data.analogsignals[data_channel].array_annotations['channel_names'][0]
                #print(channel_name)
                analog_channel_dictionary[channel_name] = channel_idx

            # ---------- Get the b2b data for the subjects
            p_fisys_data = spike2data.analogsignals[analog_channel_dictionary['p_fiSYS']] #*IMPORTANT
            p_fidia_data = spike2data.analogsignals[analog_channel_dictionary['p_fiDIA']] #*IMPORTANT
            p_physiocal_data = spike2data.analogsignals[analog_channel_dictionary['p_physiocal']] #*IMPORTANT
            p_ecg_data = spike2data.analogsignals[analog_channel_dictionary['p_ECG']] #*IMPORTANT
            c_ecg_data = spike2data.analogsignals[analog_channel_dictionary['p_ECG']] #*IMPORTANT

            #process exceptions
            if sessionID == 'BG006':
                #BG006 p and c psychophys channels are wrong way round for empathy task. Swap over!
                p_ecg_data = spike2data.analogsignals[analog_channel_dictionary['c_ECG']] #*IMPORTANT
            elif sessionID == 'BG041':
                #BG041 b2b blood pressure recorded from companion not participant
                p_ecg_data = spike2data.analogsignals[analog_channel_dictionary['c_ECG']] #*IMPORTANT


            #Prepare b2b data
            p_fisys_signal = p_fisys_data.magnitude.squeeze()
            p_fidia_signal = p_fidia_data.magnitude.squeeze()
            p_physiocal_signal = p_physiocal_data.magnitude.squeeze()
        
        # ---------- Extract textmarks from smrx file using SonPy
        smrfile = sonpy.lib.SonFile(spike_filename, True)

        textmarkChanNo = 30
        chanIdxInFile = textmarkChanNo - 1

        max_n_tick = smrfile.ChannelMaxTime(chanIdxInFile)

        n_chunks = 4
        breakpoints = divide_into_n_breakpoints(max_n_tick, n_chunks)
        df_textmarks_allchunks = pd.DataFrame()

        for i_chunk in range(n_chunks):
            first_sample = breakpoints[i_chunk]
            last_sample = breakpoints[i_chunk + 1]

            # Adjust the last chunk to ensure it includes the final tick
            if i_chunk == n_chunks - 1:
                last_sample += 1  # Add 1 to include the very last tick
            
            nSamplesToImport = last_sample - first_sample
            textmarks = smrfile.ReadTextMarks(chan=chanIdxInFile, nMax=nSamplesToImport, tFrom=first_sample, tUpto=last_sample)

            textmarks_sampleNo = np.full(len(textmarks), np.nan)
            textmarks_text = np.array(['---'] * len(textmarks), dtype=str)

            for i_mark in range(len(textmarks)):
                mark = textmarks[i_mark]
                textmarks_sampleNo[i_mark] = round(mark.Tick / 100)
                textmarks_text[i_mark] = ''.join([str(mark[0]), str(mark[1]), str(mark[2])])
            
            df_textmarks = pd.DataFrame({'sampleNo': textmarks_sampleNo, 'text': textmarks_text})
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, df_textmarks], ignore_index=True)

        #IMPORTANT - process textmark exceptions
        if sessionID == 'BG002':
            #start from row idx 19
            df_textmarks_allchunks = df_textmarks_allchunks.iloc[19:]

        elif sessionID == 'BG021':
            #add rat textmark at last sample of spike file (when it crashed)
            new_row_rating = pd.DataFrame({"sampleNo": [5509836], "text": ['rat']})  # Create a new DataFrame with the new row
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, new_row_rating], ignore_index=True)  # Append the row  

        #IMPORTANT - subtract 4 seconds from textmarks to account for transmission delay from Finometer
        df_textmarks_allchunks["sampleNo"] = df_textmarks_allchunks["sampleNo"] - 4 * SPIKE_FS

        #calculate subjectID
        subjID = sessionID + 'p'

        #Process exceptions
        if sessionID == 'BG041':
            #in session BG041, b2b data was recorded for companion not participant
            subjID = sessionID + 'c'

        #---- Define noisy sections of b2b data for exclusion
        row_p_inspection = (df_b2b_inspection['Subject'] == subjID)# Find rows in b2b inspection excel file for this subject
        row_p_ecg_inspection = (df_ecg_inspection['pID'] == subjID)# Find rows in ecg inspection excel file for this subject
        b2b_idx_noisy_section = df_b2b_inspection.loc[row_p_inspection, 'B2B_bad_segments'] # Get sample numbers of entire sections of peaks to delete

        if isinstance(b2b_idx_noisy_section, str): # Ensure it's a list or iterable of strings, in case it's a single value
            b2b_idx_noisy_section = [b2b_idx_noisy_section]

        b2b_idx_noisy_samples = [] # Initialise an empty list to collect all indices to delete

        for cell_value in b2b_idx_noisy_section: # Process each cell value
            if cell_value.lower() == 'none':  # Skip cells with 'none'
                continue
            for section in cell_value.split(','):  # Split multiple ranges by commas
                start, end = map(int, section.split(':'))  # Split each range and convert to integers
                b2b_idx_noisy_samples.extend(range(start, end + 1))  # Include the endpoint
                print(BLUE + '- Added section start=' + str(start) + " end= " + str(end) + "to b2b_idx_noisy_samples" + RESET)
                  
        b2b_noisy_samples = np.full(len(p_fisys_data), False, dtype=bool) #define logical array
        b2b_noisy_samples[b2b_idx_noisy_samples] = True #set noisy trials to true 
        
        print('- Calculated noisy samples')

        #---- Calculate when physiocal unstable
        #Define when there are <30 beats between physiocal calibrations
        p_physiocal_signal = np.round(p_physiocal_signal, 0).astype(int) #round p_physiocal_signal to 0 dp so it is either 0 or 1
        
        #Get ECG data
        p_ecg_data_np = np.squeeze(p_ecg_data)
        fps_p_ecg = int(p_ecg_data.sampling_rate.magnitude)

        #*IMPORTANT - Process exceptions
        if sessionID == "BG004":
            p_ecg_data_np.magnitude[3933300:3975500] = 0
            p_ecg_signal[3933300:3975500] = 0

        if sessionID == "BG026":
            p_ecg_data_np.magnitude[0:343500] = 0
            p_ecg_signal[0:343500] = 0

        #Detect R peaks
        _, p_ecg_peaks_forb2b = ecg_peaks(signal=p_ecg_data_np, method='sleepecg', sfreq=fps_p_ecg) # participant data - use sleepecg algorithm from Systole

        #Add missed R peaks
        idx_p_peaks_to_add = df_ecg_inspection.loc[row_p_ecg_inspection, 'add_peaks']
        idx_p_peaks_to_add = idx_p_peaks_to_add.to_numpy() #convert from series to numpy
        idx_p_peaks_to_add = np.fromstring(','.join(idx_p_peaks_to_add), sep=',', dtype=float)#convert to float
        idx_p_peaks_to_add = idx_p_peaks_to_add.astype(int)
        if len(idx_p_peaks_to_add) > 0:
            p_ecg_peaks_forb2b[idx_p_peaks_to_add] = True #change to 1 in p_ecg_peaks

        #Find runs of zeros in physiocal
        zero_runs = find_runs(p_physiocal_signal, 0)
        # Identify bad sections with <25 ECG peaks between physiocal calibrations
        sections_few_physiocals = []
        physiocal_beats_threshold = 25 #IMPORTANT - physiocal beats threshold. 
        #Set to 25 not 30 to allow for differences between cleaned ECG and finometer n
        for start, end in zero_runs:
            n_peaks = np.sum(p_ecg_peaks_forb2b[start:end])
            if n_peaks < physiocal_beats_threshold:
                sections_few_physiocals.append({
                    'start_idx': start,
                    'end_idx': end,
                    'n_peaks': n_peaks,
                    'duration_samples': end - start
                })

        #get sample number of first "apr" textmark
        apr_indices = df_textmarks_allchunks[df_textmarks_allchunks["text"] == "apr"].index
        first_apr_sample_no = int(df_textmarks_allchunks.loc[apr_indices[0], "sampleNo"])

        # Print sections with too beats between physiocals
        for section in sections_few_physiocals:
            if section['start_idx'] > first_apr_sample_no:
                print(RED + f" Unstable physiocal (<25 beats) from sample {section['start_idx']} to {section['end_idx']}: "
                    f"{section['n_peaks']} peaks (duration: {section['duration_samples']} samples)" + RESET)
                
                #Record section as unusable in b2b_noisy_samples
                b2b_noisy_samples[section['start_idx'] : section['end_idx']] = True #set samples with too few beats between physiocals to True

        #---- Interpolate short noisy sections
        # Interpolate short noisy sections
        p_idx_sections_to_interpolate = df_b2b_inspection.loc[row_p_inspection, 'B2B_interpolate'] # Get sample numbers of short sections to interpolate

        if isinstance(p_idx_sections_to_interpolate, str): # Ensure it's a list or iterable of strings, in case it's a single value
            p_idx_sections_to_interpolate = [p_idx_sections_to_interpolate]

        p_indices_to_delete = [] # Initialise an empty list to collect all indices to delete

        for cell_value in p_idx_sections_to_interpolate: # Process each cell value
            if cell_value.lower() == 'none':  # Skip cells with 'none'
                continue
            for section in cell_value.split(','):  # Split multiple ranges by commas
                start, end = map(int, section.split(':'))  # Split each range and convert to integers
                
                # Check indices are within bounds
                if end >= len(p_fisys_signal) or start < 0:
                    print(RED + "Invalid indices: start= ", str(start)," end=", str(end) + RESET)
                    continue
                
                # Indices to interpolate (exclude endpoints)
                interp_indices = np.arange(start + 1, end)
                
                # Known points (start and end of the segment)
                x_known = np.array([start, end])

                y_fisys_known = p_fisys_signal[x_known]
                y_fidia_known = p_fidia_signal[x_known]
                
                # Create cubic spline interpolator
                cs_fisys = CubicSpline(x_known, y_fisys_known)
                cs_fidia = CubicSpline(x_known, y_fidia_known)
                
                # Interpolate and replace fiSYS and fiDIA values
                p_fisys_signal[interp_indices] = cs_fisys(interp_indices)
                p_fidia_signal[interp_indices] = cs_fidia(interp_indices)

                # Print update
                print(BLUE + "Interpolated fiSYS and fiDIA data between: start= ", str(start)," end=", str(end) + RESET)
        
        #---------
        #Interpolate OR Exclude data during physiocal
        #Find runs of ones in physiocal
        one_runs = find_runs(p_physiocal_signal, 1)
        
        #For each physiocal calibration, interpolate -X s before to +X s after physiocal
        for start, end in one_runs:
            
            if how_process_physiocal == "interpolate":
                #If interpolating blood pressure values during physiocal....
                duration_extend_interpolation = 1 * SPIKE_FS #IMPORTANT - specify how long to interpolate either side of physiocal in s
                start_interpolate = start - duration_extend_interpolation #calculate interpolation start and end points
                end_interpolate = end + duration_extend_interpolation

                #make sure interpolation does not extend beyond start or end of recording
                start_interpolate = max(0, start_interpolate) #take maximum out of 0 and start_interpolate
                end_interpolate = min(len(p_physiocal_signal)-1, end_interpolate) #take minimum out of data length and end_interpolate

                # Indices to interpolate (exclude endpoints)
                interp_indices = np.arange(start_interpolate + 1, end_interpolate)
                
                # Known points (start and end of the segment)
                x_known = np.array([start_interpolate, end_interpolate])
                y_known_fisys = p_fisys_signal[x_known]
                y_known_fidia = p_fidia_signal[x_known]
                
                # Create cubic spline interpolator
                cs_fisys = CubicSpline(x_known, y_known_fisys)
                cs_fidia = CubicSpline(x_known, y_known_fidia)
                
                # Interpolate and replace fiSYS and fiDIA values
                p_fisys_signal[interp_indices] = cs_fisys(interp_indices)
                p_fidia_signal[interp_indices] = cs_fidia(interp_indices)
            
            elif how_process_physiocal == "exclude":
                #If excluding blood pressure values during physiocal....
                 b2b_noisy_samples[start:end] = True #set samples with too few beats between physiocals to True
            else:
                print(RED + "ERROR: how_process_physiocal should be either 'interpolate' or 'exclude'")

        if how_process_physiocal == "interpolate":
            # Print update
            print(BLUE + "Interpolated fiSYS and fiDIA data from ", str(duration_extend_interpolation)," ms before to ", str(duration_extend_interpolation), " ms after each physiocal" + RESET)
        elif how_process_physiocal == "exclude":
            # Print update
            print(BLUE + "Excluded fiSYS and fiDIA data from ", str(duration_extend_interpolation)," ms before to ", str(duration_extend_interpolation), " ms after each physiocal" + RESET)


        #Downsample to X Hz for plotting and median smoothing
        BP_downsample_target_fs = 10 #set target frequency for downsampling
        BP_factor_downsampled = SPIKE_FS/BP_downsample_target_fs #work out the factor by which data has been downsampled 
        p_fisys_signal_ds = nk.signal_resample(p_fisys_signal, sampling_rate = SPIKE_FS, desired_sampling_rate = BP_downsample_target_fs) #downsample fisys
        p_fidia_signal_ds = nk.signal_resample(p_fidia_signal, sampling_rate = SPIKE_FS, desired_sampling_rate = BP_downsample_target_fs) #downsample fidia
        p_fisys_times_ds = nk.signal_resample(p_fisys_data.times, sampling_rate = SPIKE_FS, desired_sampling_rate = BP_downsample_target_fs) #downsample times
        p_physiocal_signal_ds = nk.signal_resample(p_physiocal_signal, sampling_rate = SPIKE_FS, desired_sampling_rate = BP_downsample_target_fs) #downsample physiocal
        b2b_noisy_samples_ds = nk.signal_resample(b2b_noisy_samples, sampling_rate = SPIKE_FS, desired_sampling_rate = BP_downsample_target_fs) #downsample b2b_noisy_samples_ds
        
        # If median-smoothing data, then apply median smoothing
        if median_smooth_b2b_data:
            print('Starting median smoothing')
            window_size = 49  # window size for filtering in samples (must be odd)
            p_fisys_signal_ds_medsmooth = medfilt(p_fisys_signal_ds, kernel_size=window_size)
            p_fidia_signal_ds_medsmooth = medfilt(p_fidia_signal_ds, kernel_size=window_size)
            print('Completed median smoothing')

            # Define final preprocessed data as downsampled then median smoothed data
            p_fisys_preproc = p_fisys_signal_ds_medsmooth
            p_fidia_preproc = p_fidia_signal_ds_medsmooth
        else:
            # Define final preprocessed data as downsampled data
            p_fisys_preproc = p_fisys_signal_ds
            p_fidia_preproc = p_fidia_signal_ds

        #----------------Plot figure
        if plot_b2b_data:
            

            #----- Plot FiSYS data
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=p_fisys_times_ds, y=p_physiocal_signal_ds, mode='lines', name='Physiocal', line=dict(color='orange'))) #plot Physiocal trace
            fig.add_trace(go.Scatter(x=p_fisys_times_ds, y=p_fisys_signal_ds, mode='lines', name='Raw', line=dict(color='blue'))) #plot fisys trace
            
            # If median-smoothing data, then plot smoothed data
            if median_smooth_b2b_data:
                fig.add_trace(go.Scatter(x=p_fisys_times_ds, y=p_fisys_signal_ds_medsmooth, mode='lines', name='Median filt.', line=dict(color='green'))) #plot median smoothed fisys trace

            ## Add textmarks
            textmarks_of_interest = ['apr', 'rat', 'sph', 'spl', 'sch', 'scl', 'sps', 'scs'] #apr = anticipation before name displayed; rat = subjects prompted to submit rating
            df_textmarks_subset = df_textmarks_allchunks[df_textmarks_allchunks['text'].isin(textmarks_of_interest)]
            for _, row in df_textmarks_subset.iterrows():
                tm_sample_no = row['sampleNo']
                tm_text = row['text']
                
                # Add vertical line
                fig.add_vline(x=tm_sample_no/(SPIKE_FS), line_width=2, line_dash="dash", line_color="grey") #divide sample number by SPIKE_FS to convert to s
                
                # Add text annotation
                fig.add_annotation(
                    x=tm_sample_no/(SPIKE_FS),
                    y=0.5,  # Adjust this value to position the text vertically
                    text=tm_text,
                    showarrow=False,
                    textangle=-90,
                    yanchor="bottom",
                    font=dict(color="black", size=20)
                )

            fig.update_layout(
                title = f"{subjID}: FiSYS data"
            )
            
            fig.show()

            #----- Plot FiDIA data
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=p_fisys_times_ds, y=p_physiocal_signal_ds, mode='lines', name='Physiocal', line=dict(color='orange'))) #plot Physiocal trace
            fig.add_trace(go.Scatter(x=p_fisys_times_ds, y=p_fidia_signal_ds, mode='lines', name='Raw', line=dict(color='blue'))) #plot fidia trace
            
            # If median-smoothing data, then plot smoothed data
            if median_smooth_b2b_data:
                fig.add_trace(go.Scatter(x=p_fisys_times_ds, y=p_fidia_signal_ds_medsmooth, mode='lines', name='Median filt.', line=dict(color='green'))) #plot median smoothed fisys trace

            ## Add textmarks
            textmarks_of_interest = ['apr', 'rat', 'sph', 'spl', 'sch', 'scl', 'sps', 'scs'] #apr = anticipation before name displayed; rat = subjects prompted to submit rating
            df_textmarks_subset = df_textmarks_allchunks[df_textmarks_allchunks['text'].isin(textmarks_of_interest)]
            for _, row in df_textmarks_subset.iterrows():
                tm_sample_no = row['sampleNo']
                tm_text = row['text']
                
                # Add vertical line
                fig.add_vline(x=tm_sample_no/(SPIKE_FS), line_width=2, line_dash="dash", line_color="grey") #divide sample number by SPIKE_FS to convert to s
                
                # Add text annotation
                fig.add_annotation(
                    x=tm_sample_no/(SPIKE_FS),
                    y=0.5,  # Adjust this value to position the text vertically
                    text=tm_text,
                    showarrow=False,
                    textangle=-90,
                    yanchor="bottom",
                    font=dict(color="black", size=20)
                )

            fig.update_layout(
                title = f"{subjID}: FiDIA data"
            )
            
            fig.show()

        #Multiply b2b blood pressure values by 100 to make them correct scale
        p_fisys_signal_ds = p_fisys_signal_ds * 100
        p_fidia_signal_ds = p_fidia_signal_ds * 100
        
        #-------- Identify textmarks for each trial
 
        #check that number of anticipation and shock textmarks matches number of trials
        nTrialsInTextmarks = len(df_textmarks_allchunks[df_textmarks_allchunks['text'].isin(['apr'])]) #get number of trials in textmarks
        subj_rows_combined = df_all_combined['subjectID'].str.contains(subjID, na=False, case=False)  #get subject rows in df_all_combined

        if nTrialsInTextmarks != sum(subj_rows_combined):
            print(RED + 'ERROR: there are ' + str(nTrialsInTextmarks) + 
                ' apr textmarks but ' + str(sum(subj_rows_combined)) + 
                ' apr textmarks but ' + str(sum(subj_rows_combined)) + 
                ' rows of data for this subject in df_all_combined' + RESET)
        else:
            print('- No. of apr textmarks (' + str(nTrialsInTextmarks) + 
                ') matches no. rows (' + str(sum(subj_rows_combined)) + 
                ') data for this subject in df_all_combined')


        # Iterate through each trial and calculate SCL/SCR measures for each phase
        #> Filter rows for the relevant textmarks in df_textmarks_all_chunks
        relevant_textmarks = ["apr", #anticipation pre-name
                                "apo", #anticipation post-name
                                "sph", "spl", "sps", "sch", "scl", "scs", #shock delivery
                                "res", #start of response period
                                "rat"] #submit rating/experimenter change shock settings
        filtered_textmarks = df_textmarks_allchunks[df_textmarks_allchunks["text"].isin(relevant_textmarks)]

        # Find all indices of the "apr" textmark (one for each trial)
        apr_indices = filtered_textmarks[filtered_textmarks["text"] == "apr"].index


        # Iterate through each trial using "apr" textmarks as the starting point
        for trial_idx, apr_idx in enumerate(apr_indices):
            
            #Calculate trial number (start at 1 not 0)
            trial_no = trial_idx + 1
            
            #calculate subject row as logical array
            subject_row = (df_all_combined['subjectID'].str.contains(subjID, na=False, case=False)) & (df_all_combined['trialNo'] == trial_no)

            # Get the indices of the textmarks for this trial
            sample_no_apr = int(filtered_textmarks.loc[apr_idx, "sampleNo"])

            # Get the next "apo" textmark (signalling start of anticipation post name period) after "apr"
            sample_no_apo = filtered_textmarks.loc[
                (filtered_textmarks.index > apr_idx) & (filtered_textmarks["text"] == "apo"), "sampleNo"
            ].iloc[0]

            # Get the first shock (sph, spl, sps, sch, scl, scs) textmark after "apo"
            sample_no_shock = filtered_textmarks.loc[
                (filtered_textmarks.index > apr_idx)
                & (filtered_textmarks["text"].isin(["sph", "spl", "sps", "sch", "scl", "scs"])),
                "sampleNo",
            ].iloc[0]

            #Get the first response period (res) textmark after "apo"
            sample_no_res = filtered_textmarks.loc[
                (filtered_textmarks.index > apr_idx) & (filtered_textmarks["text"] == "res"), "sampleNo"
            ].iloc[0]

            # Get the next "rat" textmark (signalling rating) after anticipation post name
            sample_no_rat = filtered_textmarks.loc[
                (filtered_textmarks.index > apr_idx)
                & (filtered_textmarks["text"] == "rat"),
                "sampleNo",
            ].iloc[0]

            #Convert indices to integers
            sample_no_apr_ds = int(round(int(sample_no_apr)/BP_factor_downsampled, 0))
            sample_no_apo_ds = int(round(int(sample_no_apo)/BP_factor_downsampled, 0))
            sample_no_shock_ds = int(round(int(sample_no_shock)/BP_factor_downsampled, 0))
            sample_no_res_ds = int(round(int(sample_no_res)/BP_factor_downsampled, 0))
            sample_no_rat_ds = int(round(int(sample_no_rat)/BP_factor_downsampled, 0))
            durationShock_ds = int(round(DURATION_SHOCK/BP_factor_downsampled, 0))
            durationWaiting_ds = int(round(DURATION_WAITING/BP_factor_downsampled, 0))
            durationResponseExclShock_ds = int(round(DURATION_RESPONSE_EXCL_SHOCK/BP_factor_downsampled, 0))
            durationPrePhase_ds = int(round(DURATION_PRE_PHASE/BP_factor_downsampled, 0))

            #Check anticipation phase textmarks
            if (sample_no_shock_ds - sample_no_apo_ds > round(10100/BP_factor_downsampled, 0) ) | (sample_no_shock_ds - sample_no_apo_ds < round(5900/BP_factor_downsampled, 0) ):
                #print(RED + '- Trial ' +  str(trial_no) + ', Anticipation Phase is ' + str(sample_no_shock - sample_no_apo) + ' ms' + RESET)
                
                #If antiicpation phase is incorrect length, work out shock time as 500ms before start of response phase
                sample_no_shock_ds = sample_no_res_ds - durationShock_ds
                #print(BLUE + '- Trial ' +  str(trial_no) + ', New Anticipation Phase duration: ' + str(sample_no_shock - sample_no_apo) + ' ms' + RESET)
                
                #if this does not fix it then print error
                if (sample_no_shock_ds - sample_no_apo_ds > round(10100/BP_factor_downsampled,0) ) | (sample_no_shock_ds - sample_no_apo_ds < round(5900/BP_factor_downsampled,0) ):
                    print(RED + '- ERROR: Trial ' +  str(trial_no) + ', Anticipation Phase is ' + str(sample_no_shock_ds - sample_no_apo_ds) + ' ms even after using "res" textmark.' + RESET)

            #-------- Calculate B2B measures for each phase

            #Waiting
            if any(b2b_noisy_samples_ds[sample_no_apr_ds : sample_no_apr_ds + durationWaiting_ds]): #if eda data is noisy then set values to NaN
                    mean_fisys_waiting = np.nan # mean fisys
                    mean_fidia_waiting = np.nan  # mean fidia
            else:
                p_fisys_waiting = p_fisys_signal_ds[sample_no_apr_ds : sample_no_apr_ds + durationWaiting_ds] 
                p_fidia_waiting = p_fidia_signal_ds[sample_no_apr_ds : sample_no_apr_ds + durationWaiting_ds] 
                mean_fisys_waiting = p_fisys_waiting.mean() # Mean fiSYS
                mean_fidia_waiting = p_fidia_waiting.mean() # Mean fiDIA


            #Anticipation
            if any(b2b_noisy_samples_ds[sample_no_apo_ds : sample_no_shock_ds]): #if eda data is noisy then set values to NaN
                    mean_fisys_anticipation = np.nan # mean fisys
                    mean_fidia_anticipation = np.nan  # mean fidia
            else:
                p_fisys_anticipation = p_fisys_signal_ds[sample_no_apo_ds : sample_no_shock_ds] 
                p_fidia_anticipation = p_fidia_signal_ds[sample_no_apo_ds : sample_no_shock_ds] 
                mean_fisys_anticipation = p_fisys_anticipation.mean() # Mean fiSYS
                mean_fidia_anticipation = p_fidia_anticipation.mean()  # Mean fiDIA

            #Response
            if any(b2b_noisy_samples_ds[sample_no_shock_ds + durationShock_ds : sample_no_shock_ds + durationShock_ds + durationResponseExclShock_ds]): #if eda data is noisy then set values to NaN
                    mean_fisys_response = np.nan # mean fisys
                    mean_fidia_response = np.nan  # mean fidia
            else:
                p_fisys_response = p_fisys_signal_ds[sample_no_shock_ds + durationShock_ds : sample_no_shock_ds + durationShock_ds + durationResponseExclShock_ds] 
                p_fidia_response = p_fidia_signal_ds[sample_no_shock_ds + durationShock_ds : sample_no_shock_ds + durationShock_ds + durationResponseExclShock_ds] 
                #do not include shock in mean
                mean_fisys_response = p_fisys_response.mean() # Mean fiSYS
                mean_fidia_response = p_fidia_response.mean()  # Mean fiDIA
                
            #------------ Calculate b2b timeseries for each phase
            
            #Waiting
            if any(b2b_noisy_samples_ds[sample_no_apr_ds - durationPrePhase_ds : sample_no_apr_ds + durationWaiting_ds]): #if eda data is noisy then set values to NaN
                downsampled_fisys_waiting_ts = np.array([np.nan])
                downsampled_fidia_waiting_ts = np.array([np.nan])
            else:
                p_fisys_waiting_ts = p_fisys_signal_ds[sample_no_apr_ds - durationPrePhase_ds : sample_no_apr_ds + durationWaiting_ds]
                p_fidia_waiting_ts = p_fidia_signal_ds[sample_no_apr_ds - durationPrePhase_ds : sample_no_apr_ds + durationWaiting_ds]
                downsampled_fisys_waiting_ts = np.round(downsample_with_last(p_fisys_waiting_ts, BP_downsample_target_fs, target_fs = target_fs_ts), 2) #downsample
                downsampled_fidia_waiting_ts = np.round(downsample_with_last(p_fidia_waiting_ts, BP_downsample_target_fs, target_fs = target_fs_ts), 2) #downsample

            #Anticipation
            if any(b2b_noisy_samples_ds[sample_no_apo_ds - durationPrePhase_ds : sample_no_shock_ds]): #if eda data is noisy then set values to NaN
                downsampled_fisys_anticipation_ts = np.array([np.nan])
                downsampled_fidia_anticipation_ts = np.array([np.nan])
            else:
                p_fisys_anticipation_ts = p_fisys_signal_ds[sample_no_apo_ds - durationPrePhase_ds : sample_no_shock_ds]
                p_fidia_anticipation_ts = p_fidia_signal_ds[sample_no_apo_ds - durationPrePhase_ds : sample_no_shock_ds]
                downsampled_fisys_anticipation_ts = np.round(downsample_with_last(p_fisys_anticipation_ts, BP_downsample_target_fs, target_fs = target_fs_ts), 2) #downsample
                downsampled_fidia_anticipation_ts = np.round(downsample_with_last(p_fidia_anticipation_ts, BP_downsample_target_fs, target_fs = target_fs_ts), 2) #downsample

            #Response
            if any(b2b_noisy_samples_ds[sample_no_shock_ds - durationPrePhase_ds : sample_no_shock_ds + durationShock_ds + durationResponseExclShock_ds]): #if eda data is noisy then set values to NaN
                downsampled_fisys_response_ts = np.array([np.nan])
                downsampled_fidia_response_ts = np.array([np.nan])
            else:
                #include shock in timeseries
                p_fisys_response_ts = p_fisys_signal_ds[sample_no_shock_ds - durationPrePhase_ds : sample_no_shock_ds + durationShock_ds + durationResponseExclShock_ds]
                p_fidia_response_ts = p_fidia_signal_ds[sample_no_shock_ds - durationPrePhase_ds : sample_no_shock_ds + durationShock_ds + durationResponseExclShock_ds]
                downsampled_fisys_response_ts = np.round(downsample_with_last(p_fisys_response_ts, BP_downsample_target_fs, target_fs = target_fs_ts), 2) #downsample
                downsampled_fidia_response_ts = np.round(downsample_with_last(p_fidia_response_ts, BP_downsample_target_fs, target_fs = target_fs_ts), 2) #downsample

            #if subject_row is valid then add fisys and fidia values to dataframe
            if sum(subject_row) == 1:
                # Add mean fisys to dataframe
                df_all_combined.loc[subject_row, "mean_fisys_waiting"] = np.round(mean_fisys_waiting, 4) 
                df_all_combined.loc[subject_row, "mean_fisys_anticipation"] = np.round(mean_fisys_anticipation, 4)
                df_all_combined.loc[subject_row, "mean_fisys_response"] = np.round(mean_fisys_response, 4)

                # Add mean fidia to dataframe
                df_all_combined.loc[subject_row, "mean_fidia_waiting"] = np.round(mean_fidia_waiting, 4)
                df_all_combined.loc[subject_row, "mean_fidia_anticipation"] = np.round(mean_fidia_anticipation, 4)
                df_all_combined.loc[subject_row, "mean_fidia_response"] = np.round(mean_fidia_response, 4)

                # Add fisys timeseries to the dataframe
                #>Assign as a string representation of the list
                df_all_combined.loc[subject_row, 'fisys_series_waiting'] = str(downsampled_fisys_waiting_ts.tolist())
                df_all_combined.loc[subject_row, 'fisys_series_anticipation'] = str(downsampled_fisys_anticipation_ts.tolist())
                df_all_combined.loc[subject_row, 'fisys_series_response'] = str(downsampled_fisys_response_ts.tolist())

                # Add fisys timeseries to the dataframe
                #>Assign as a string representation of the list
                df_all_combined.loc[subject_row, 'fidia_series_waiting'] = str(downsampled_fidia_waiting_ts.tolist())
                df_all_combined.loc[subject_row, 'fidia_series_anticipation'] = str(downsampled_fidia_anticipation_ts.tolist())
                df_all_combined.loc[subject_row, 'fidia_series_response'] = str(downsampled_fidia_response_ts.tolist())

                #Process exceptions

                #> BG021 trial 79 crashed during response phase so set responses to nan
                if (sessionID == 'BG021') & (trial_no == 79):
                    df_all_combined.loc[subject_row, "mean_fisys_response"] = np.nan
                    df_all_combined.loc[subject_row, "mean_fidia_response"] = np.nan
                    df_all_combined.loc[subject_row, 'fisys_series_response'] = str( np.array([np.nan]) )
                    df_all_combined.loc[subject_row, 'fidia_series_response'] = str( np.array([np.nan]) )
                    print(BLUE + 'Set fisys/fidia responses for trial 79 response phase to nan' + RESET)

            elif sum(subject_row) > 1 :
                print(RED + f"WARNING: multiple subject_rows {subject_row} is in df_all_combined." + RESET)
            elif sum(subject_row) == 0 :
                print(RED + f"WARNING: subject_row {subject_row} is not in df_all_combined." + RESET)
        
        #Process exceptions
        #> BG001p b2b data is unusable so do not include any b2b data for any trials
        if subjID == 'BG001p':
            df_all_combined.loc[subj_rows_combined, ["mean_fisys_waiting", "mean_fisys_anticipation", "mean_fisys_response"]] = np.nan
            df_all_combined.loc[subj_rows_combined, ["mean_fidia_waiting", "mean_fidia_anticipation", "mean_fidia_response"]] = np.nan
            df_all_combined.loc[subj_rows_combined, ['fisys_series_waiting', 'fisys_series_anticipation', 'fisys_series_response',
                                                     'fidia_series_waiting', 'fidia_series_anticipation', 'fidia_series_response']] = str( np.array([np.nan]) )
            print(BLUE + '- BG007p set b2b responses for all trials nan due to unusable b2b data' + RESET)

        #Print message
        print('- Successfully calculated fisys/fidia resposnes')                


Session:  BG001
- Importing Spike data for BG001
- Calculated noisy samples
Interpolated fiSYS and fiDIA data from  1000  ms before to  1000  ms after each physiocal
Starting median smoothing
Completed median smoothing
- No. of apr textmarks (80) matches no. rows (80) data for this subject in df_all_combined
- BG007p set b2b responses for all trials nan due to unusable b2b data
- Successfully calculated fisys/fidia resposnes
Session:  BG002
- Importing Spike data for BG002
- Calculated noisy samples
Interpolated fiSYS and fiDIA data between: start=  3297200  end= 3299237
Interpolated fiSYS and fiDIA data between: start=  3508922  end= 3509872
Interpolated fiSYS and fiDIA data from  1000  ms before to  1000  ms after each physiocal
Starting median smoothing
Completed median smoothing
- No. of apr textmarks (40) matches no. rows (40) data for this subject in df_all_combined
- Successfully calculated fisys/fidia resposnes
Session:  BG003
- Importing Spike data for BG003
- Added section st

Calculate resting state HRV and EDA measures

In [ ]:
run_rs_analyses = True #define whether to run this section of skip it
# Visual verification variables
hr_visual_verification = False
eda_visual_verification = False


if run_rs_analyses:
    # Suppress the specific DeprecationWarning
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    num_plotted = 0
    max_plotted = 100
    ecg_visual_verif_done = ['BG001', 'BG002', 'BG003', 'BG004', 'BG005',
                             'BG006', 'BG007', 'BG008', 'BG009', 'BG010',
                             'BG011', 'BG012', 'BG013', 'BG014', 'BG015',
                             'BG016', 'BG017', 'BG018', 'BG019', 'BG020',
                             'BG021', 'BG022', 'BG023', 'BG024', 'BG025',
                             'BG026', 'BG027', 'BG028', 'BG029', 'BG030',
                             'BG031', 'BG032', 'BG033', 'BG034', 'BG035',
                             'BG036', 'BG037', 'BG038', 'BG039', 'BG040',
                             'BG041', 'BG042', 'BG043', 'BG044', 'BG045',
                             'BG046', 'BG047', 'BG048', 'BG049', 'BG050']

    #------- Peak detection algorithm exceptions
    peak_detection_algs = {sessionID: 'sleepecg' for sessionID in session_folders}
    #peak_detection_algs['BG026'] = 'hamilton' #option to swap peak detection algorithm to 'hamilton' for selected participants if 'sleepecg' algorithm

    #------- Import excel file containing notes from ecg inspection
    analysisFolder = BASE_DIR / 'Analysis'
    rs_ecg_inspection_file = os.path.join(analysisFolder, 'Data checks', 'rs_ecg_inspection.xlsx')
    df_rs_ecg_inspection = pd.read_excel(rs_ecg_inspection_file)

    # ------- Dictionary for resting-state textmark exceptions
    rs_exceptions = {
        'BG002': {'start': 418900, 'end': 418900 + 120000},
        'BG016': {'start': 'rss_last', 'end': 152249},
        'BG023': {'start': 213083, 'end': 333091},
        'BG028': {'start': 200131, 'end': 320135},
        'BG029': {'start': 417083, 'end': 537067},
        'BG030': {'start': 519183, 'end': 639222},
        'BG035': {'start': 3262,   'end': 123294},
        'BG036': {'start': 177003, 'end': 297034},
        'BG037': {'start': 34779,  'end': 154791},
        'BG038': {'start': 41609,  'end': 161617},
        'BG039': {'start': 38219,  'end': 158253},
        'BG040': {'start': 31694,  'end': 151746},
        'BG041': {'start': 26406,  'end': 146410},
        'BG042': {'start': 214415, 'end': 334427},
        'BG043': {'start': 114942, 'end': 234954},
        'BG044': {'start': 37418,  'end': 157438},
        'BG045': {'start': 94772,  'end': 214796},
        'BG047': {'start': 30088,  'end': 150118},
        'BG048': {'start': 10695,  'end': 130675},
        'BG049': {'start': 37199,  'end': 157227},
        'BG050': {'start': 682066, 'end': 802089},
    }

    for sessionID in session_folders:

        # get session folder
        sessionFolder = os.path.join(directory, sessionID)
        print('Session: ', sessionID) 
        
        #debugging:
        #print(sessionFolder)

        # Look for spike data file ending rs.smrx
        rs_spike_file_extension = "rs.smrx"
        rs_spike_search_pattern = os.path.join(sessionFolder, f'*{rs_spike_file_extension}')
        rs_spike_files = glob.glob(rs_spike_search_pattern)
        print(rs_spike_files)

        #if found then use it, otherwise use empathy.smrx file
        if len(rs_spike_files) >= 1:
            spike_filename = rs_spike_files[0]
            print("-", "rs.smrx file found")
        else:
            empathy_spike_file_extension = "empathy.smrx"
            rs_spike_search_pattern = os.path.join(sessionFolder, f'*{empathy_spike_file_extension}')
            rs_spike_files = glob.glob(rs_spike_search_pattern)
            spike_filename = rs_spike_files[0]
            print("-", "using empathy.smrx spike file")

        #Process exceptions
        if sessionID == 'BG042':
            spike_filename = BASE_DIR / "Participant data" / "BG042" / "BG042_rs_duplicate.smrx"

        if sessionID == 'BG043':
            spike_filename = BASE_DIR / "Participant data" / "BG043" / "BG043_rs_duplicate.smrx"

        # Loop over files
        print("-", "Importing Spike data for {}".format(sessionID))

        # Read Spike2 file
        reader = CedIO(filename=spike_filename)
        
        block = reader.read_block()
        spike2data = block.segments[0] # single segment

        # Create a dictionary of analog channels and their indexes
        analog_channel_dictionary = {}

        for data_channel in range(len(spike2data.analogsignals)):
            channel_idx = data_channel
            channel_name = spike2data.analogsignals[data_channel].array_annotations['channel_names'][0]
            analog_channel_dictionary[channel_name] = channel_idx

        #print('Analog channels identified:', analog_channel_dictionary)
    
        # ---------- Extract textmarks from smrx file using SonPy
        smrfile = sonpy.lib.SonFile(spike_filename, True)

        textmarkChanNo = 30
        chanIdxInFile = textmarkChanNo - 1

        max_n_tick = smrfile.ChannelMaxTime(chanIdxInFile)
        #n_samples = int(np.floor(max_n_tick))
        #print('Max n tick =', max_n_tick)

        n_chunks = 4
        breakpoints = divide_into_n_breakpoints(max_n_tick, n_chunks)
        df_textmarks_allchunks = pd.DataFrame()

        for i_chunk in range(n_chunks):
            first_sample = breakpoints[i_chunk]
            last_sample = breakpoints[i_chunk + 1]

            # Adjust the last chunk to ensure it includes the final tick
            if i_chunk == n_chunks - 1:
                last_sample += 1  # Add 1 to include the very last tick
            
            nSamplesToImport = last_sample - first_sample
            textmarks = smrfile.ReadTextMarks(chan=chanIdxInFile, nMax=nSamplesToImport, tFrom=first_sample, tUpto=last_sample)
            
            textmarks_sampleNo = np.full(len(textmarks), np.nan)
            textmarks_text = np.array(['---'] * len(textmarks), dtype=str)

            for i_mark in range(len(textmarks)):
                mark = textmarks[i_mark]
                textmarks_sampleNo[i_mark] = round(mark.Tick / 100)
                textmarks_text[i_mark] = ''.join([str(mark[0]), str(mark[1]), str(mark[2])])
            
            df_textmarks = pd.DataFrame({'sampleNo': textmarks_sampleNo, 'text': textmarks_text})
            df_textmarks_allchunks = pd.concat([df_textmarks_allchunks, df_textmarks], ignore_index=True)

            #Find textmarks for start and end of resting state recording
            if sessionID in rs_exceptions:
                exception = rs_exceptions[sessionID]
                if exception['start'] == 'rss_last':
                    tm_rs_start = df_textmarks_allchunks.loc[
                        df_textmarks_allchunks['text'] == 'rss', 'sampleNo'].iloc[-1] # take final rss textmark
                else:
                    tm_rs_start = exception['start']
                tm_rs_end = exception['end']
            else:
                #For all other sessions:
                tm_rs_start = df_textmarks_allchunks.loc[
                    df_textmarks_allchunks['text'] == 'rss', 'sampleNo'].iloc[-1] # take final rss textmark
                tm_rs_end = df_textmarks_allchunks.loc[
                    df_textmarks_allchunks['text'] == 'rse', 'sampleNo'].iloc[-1] # take final rse textmark
            
            #Check textmarks found
            if pd.isna(tm_rs_start):
                print(RED + 'WARNING: no rss textmark found')
            elif pd.isna(tm_rs_end):
                print(RED + 'WARNING: no rse textmark found')

            #Check resting state is correct length
            rs_tm_duration = tm_rs_end - tm_rs_start
            rs_desired_duration = 120 * SPIKE_FS
            if abs(rs_desired_duration - rs_tm_duration) > 5*SPIKE_FS : #+- 5s is acceptable
                print(RED + 'ERROR: resting state duration = ' + str( round( ( (tm_rs_end - tm_rs_start)/SPIKE_FS), 2) ) + ' s' + RESET )
        
        
        #-----------Get EDA scaling factors
        #calculate participant scaling factor
        p_id = sessionID + 'p'
        row1_p_combined = df_all_combined.index[df_all_combined["subjectID"].str.lower() == p_id.lower()][0]
        p_eda_range = df_all_combined.loc[row1_p_combined, 'eda_range'] #get input range
        if isinstance(p_eda_range, pd.Series):
            # If 'p_eda_range' is still a series with multiple rows, retrieve the first element
            p_eda_range = p_eda_range.iloc[0]  # Get the first value
        p_transferConstant = p_eda_range/dataRange #calculate participant transfer constant
        print('- p transfer constant = ' + str(p_transferConstant))

        #calculate companion scaling factor
        c_id = sessionID + 'c'
        row1_c_combined = df_all_combined.index[df_all_combined["subjectID"].str.lower() == c_id.lower()][0]
        c_eda_range = df_all_combined.loc[row1_c_combined, 'eda_range'] #get input range
        if isinstance(c_eda_range, pd.Series):
            # If 'c_eda_range' is still a series with multiple rows, retrieve the first element
            c_eda_range = c_eda_range.iloc[0]  # Get the first value
        c_transferConstant = c_eda_range/dataRange #calculate companion transfer constant
        print('- c transfer constant = ' + str(c_transferConstant))
 
        #generate IDs of both subjects in session
        subjsInSession = ["p", "c"]

        for subj in subjsInSession:
            
            #------------- Import ECG+EDA data
            subjID = sessionID + subj

            if subj == "p": #IMPORTANT - defines array for calculating HR responses
                #*IMPORTANT - import ECG and EDA data
                subj_ecg_data = spike2data.analogsignals[analog_channel_dictionary['p_ECG']] 
                subj_eda_data = spike2data.analogsignals[analog_channel_dictionary['p_EDA']] #*IMPORTANT
                subj_transfer_constant = p_transferConstant
                
            elif subj == "c":
                subj_ecg_data = spike2data.analogsignals[analog_channel_dictionary['c_ECG']] #*IMPORTANT
                subj_eda_data = spike2data.analogsignals[analog_channel_dictionary['c_EDA']] #*IMPORTANT
                subj_transfer_constant = c_transferConstant
            else:
                print(RED + 'ERROR - issue defining subj_ecg_data - subj not recognised')

            #Process exceptions
            if subjID == 'BG006p':
                #BG006 p and c psychophys channels are wrong way round for resting state. Swap over!
                subj_ecg_data = spike2data.analogsignals[analog_channel_dictionary['c_ECG']] 
                subj_eda_data = spike2data.analogsignals[analog_channel_dictionary['c_EDA']] #*IMPORTANT
                subj_transfer_constant = c_transferConstant
            elif subjID == 'BG006c':
                subj_ecg_data = spike2data.analogsignals[analog_channel_dictionary['p_ECG']] 
                subj_eda_data = spike2data.analogsignals[analog_channel_dictionary['p_EDA']] #*IMPORTANT
                subj_transfer_constant = p_transferConstant

            #--------Process ecg data--------------------------------------------------------------
            subj_ecg_data_np = np.squeeze(subj_ecg_data)
            subj_ecg_signal = subj_ecg_data.magnitude.squeeze()
            fps_subj_ecg = int(subj_ecg_data.sampling_rate.magnitude)

            #Process exceptions
            if subjID == "BG026p":
                subj_ecg_data_np.magnitude[0:343500] = 0
                subj_ecg_signal[0:343500] = 0

            #----- Identify and correct R peaks
            # Detect R peaks *IMPORTANT* lines
            _, subj_ecg_peaks = ecg_peaks(signal=subj_ecg_data_np, method=peak_detection_algs[sessionID], sfreq=fps_subj_ecg) # participant data - use sleepecg algorithm from Systole

            #df_textmarks_allchunks.to_csv('df_textmarks_allchunks.csv')

            if hr_visual_verification:
                plot_ECG_peaks(sessionID, subj, subj_ecg_data, subj_ecg_peaks, subj_ecg_signal, df_textmarks_allchunks, resting_or_main = 'resting')
    
            #------ Delete entire sections of mislabelled peaks-------
            print(sessionID + subj + ' peak deletion/addition')

            # Find rows in ecg inspection excel file for this recording from this participant
            row_subj_recording = (df_rs_ecg_inspection['pID'] == sessionID + subj)

            # Delete entire sections of peaks (for replacing slightly but acceptably noisy sections with specific peaks listed in add_peaks)
            subj_idx_sections_to_delete_peaks = df_rs_ecg_inspection.loc[row_subj_recording, 'sections_all_peaks_zero'] # Get sample numbers of entire sections of peaks to delete

            if isinstance(subj_idx_sections_to_delete_peaks, str): # Ensure it's a list or iterable of strings, in case it's a single value
                subj_idx_sections_to_delete_peaks = [subj_idx_sections_to_delete_peaks]

            subj_indices_to_delete = [] # Initialise an empty list to collect all indices to delete

            for cell_value in subj_idx_sections_to_delete_peaks: # Process each cell value
                if cell_value.lower() == 'none':  # Skip cells with 'none'
                    continue
                for section in cell_value.split(','):  # Split multiple ranges by commas
                    start, end = map(int, section.split(':'))  # Split each range and convert to integers
                    subj_indices_to_delete.extend(range(start, end + 1))  # Include the endpoint

            if subj_indices_to_delete: # Convert to a numpy array for assignment if there are indices to delete
                subj_indices_to_delete = np.array(subj_indices_to_delete, dtype=int)
                # Delete all peaks in the specified sections
                subj_ecg_peaks[subj_indices_to_delete] = 0  # Set the values to zero
                print(
                    BLUE + '-',
                    'Successfully deleted peaks from entire ranges:',
                    ', '.join(subj_idx_sections_to_delete_peaks),
                    RESET
                )

            #------Delete specific mislabelled peaks-------
            # Get sample numbers of peaks to delete
            idx_subj_peaks_to_delete = df_rs_ecg_inspection.loc[row_subj_recording, 'delete_peaks']
            idx_subj_peaks_to_delete = idx_subj_peaks_to_delete.to_numpy() #convert from series to numpy
            idx_subj_peaks_to_delete = np.fromstring(','.join(idx_subj_peaks_to_delete), sep=',', dtype=float)#convert to float
            idx_subj_peaks_to_delete = idx_subj_peaks_to_delete.astype(int)

            if len(idx_subj_peaks_to_delete) == 0:
                print('-', 'No peaks deleted')

            else:
                #check that there is a peak at this sample number (i.e. that the value in subj_ecg_peaks at that sample is 1)
                subj_peak_vals = subj_ecg_peaks[idx_subj_peaks_to_delete]
                subj_mislabelled_peaks = idx_subj_peaks_to_delete[subj_peak_vals == 0] #get sample numbers where there is no peak to delete
                subj_truelabelled_peaks = idx_subj_peaks_to_delete[subj_peak_vals == 1] #get sample numbers where there is indeed a peak to delete

                #Delete peaks
                if len(subj_truelabelled_peaks) > 0:
                    #subj_ecg_peaks[subj_truelabelled_peaks] = False #change from one to zero in subj_ecg_peaks_all_chunks to delete peak
                    print(BLUE + '-', 'Successfully deleted peaks at',  str(subj_truelabelled_peaks) + RESET)

                # Print warning if there is no peak at the sample no. then print warning
                if len(subj_mislabelled_peaks) > 0:
                    print(RED + 'ERROR: No peaks to delete at sample nos. ', subj_mislabelled_peaks, RESET) #print message

                    #For some participants the sample numbers are shifted by 1 so check if there are peaks on the previous samples
                    subj_peak_vals_min1 = subj_ecg_peaks[idx_subj_peaks_to_delete - 1]
                    subj_mislabelled_peaks_min1 = idx_subj_peaks_to_delete[subj_peak_vals_min1 == 0]
                    #or next samples
                    subj_peak_vals_plus1 = subj_ecg_peaks[idx_subj_peaks_to_delete + 1]
                    subj_mislabelled_peaks_plus1 = idx_subj_peaks_to_delete[subj_peak_vals_plus1 == 0]

                    if any(subj_peak_vals_min1) == 1:
                        #If all peaks are present 1 sample previously then process
                        print(BLUE + 'However peaks found on previous samples ' + str(idx_subj_peaks_to_delete[subj_peak_vals_min1 == 1]-1) + RESET) 

                        if len(subj_mislabelled_peaks_min1 > 0 ):
                            print(RED + 'However no peaks found at ' + str(subj_mislabelled_peaks_min1 -1) + RESET)                        

                    if any(subj_peak_vals_plus1) == 1:
                        #If any peaks are present 1 sample subsequently then process
                        print(BLUE + 'However peaks found on next samples ' + str(idx_subj_peaks_to_delete[subj_peak_vals_plus1 == 1]) + RESET) 

                        if len(subj_mislabelled_peaks_plus1 > 0 ):
                            print(RED + 'However no peaks found at ' + str(subj_mislabelled_peaks_plus1) + RESET)

 
            #Delete peaks on sample, previous sample, and next sample to be sure
            subj_ecg_peaks[idx_subj_peaks_to_delete] = False
            subj_ecg_peaks[idx_subj_peaks_to_delete - 1] = False
            subj_ecg_peaks[idx_subj_peaks_to_delete + 1] = False

            #----------- Add missed peaks---------------------
            # Get sample numbers of peaks to add
            idx_subj_peaks_to_add = df_rs_ecg_inspection.loc[row_subj_recording, 'add_peaks']
            idx_subj_peaks_to_add = idx_subj_peaks_to_add.to_numpy() #convert from series to numpy
            idx_subj_peaks_to_add = np.fromstring(','.join(idx_subj_peaks_to_add), sep=',', dtype=float)#convert to float
            idx_subj_peaks_to_add = idx_subj_peaks_to_add.astype(int)

            #add peak to subj_ecg_peaks
            if len(idx_subj_peaks_to_add) > 0:
                subj_ecg_peaks[idx_subj_peaks_to_add] = 1 #change to 1 in subj_ecg_peaks
                print(BLUE + '-', 'Added missing peaks ', str(idx_subj_peaks_to_add) + RESET)

            #----------calculate instantaneous HR---------------------
            #Calculate instantaneous HR from subj_ecg_peaks
            subj_inst_hr = calculate_instantaneous_hr(subj_ecg_peaks, SPIKE_FS)            

            #-----------------Calculate HRV parameters

            # --- Interpolate RR intervals containing ectopic beats before HRV analysis ---
            idx_subj_ectopics_to_interpolate = df_rs_ecg_inspection.loc[row_subj_recording, 'peak_to_interpolate']
            idx_subj_ectopics_to_interpolate = idx_subj_ectopics_to_interpolate.to_numpy() #convert from series to numpy
            idx_subj_ectopics_to_interpolate = np.fromstring(','.join(idx_subj_ectopics_to_interpolate), sep=',', dtype=float)#convert to float
            idx_subj_ectopics_to_interpolate = idx_subj_ectopics_to_interpolate.astype(int)

            # Get logical array of ecg peaks between resting state textmarks
            rs_subj_ecg_peaks = subj_ecg_peaks[int(tm_rs_start) : int(tm_rs_end)]
            
            # Subtract tm_rs_start from values in idx_subj_ectopics_to_interpolate to get the indices relative to the start of the resting state
            rs_idx_peaks_to_delete = idx_subj_ectopics_to_interpolate - int(tm_rs_start)  # Indices of ectopic beats to interpolate
            rs_idx_peaks_to_delete = rs_idx_peaks_to_delete.astype(int)  # Convert to integer type

            #debugging
            print('idx_subj_ectopics_to_interpolate', idx_subj_ectopics_to_interpolate)
            print('rs_idx_peaks_to_delete', rs_idx_peaks_to_delete)

            def interpolate_ectopic_rr_cubic_resting(ecg_peaks, idx_peaks_to_interpolate):
                
                """
                Interpolate R-R intervals containing ectopic beats using cubic spline interpolation.

                Parameters
                ----------
                ecg_peaks : array-like (bool)
                    Logical array of detected R-peaks in the ECG signal (True for peak, False otherwise).
                idx_peaks_to_interpolate : array-like (int)
                    Indices of ectopic R-peaks that should be interpolated.

                Returns
                -------
                rs_rr_intervals_samples : np.ndarray
                    Array of R-peak sample indices for the resting-state period, with ectopic RR intervals interpolated.

                Notes
                -----
                - Uses cubic spline interpolation on valid R-R intervals, excluding ectopic beats.
                - Prints debug messages showing original and interpolated IBI values.
                - Requires at least 4 valid intervals for spline fitting; otherwise prints an error.
                """

                #Get the R-peak locations in the resting state
                rs_rpeaks_samples = np.where(ecg_peaks)[0]  # Indices of detected R-peaks

                # Get R-R intervals
                rs_rr_intervals = np.diff(rs_rpeaks_samples)  # Calculate R-R intervals

                for ectopic in idx_peaks_to_interpolate:
                    # Get the index of the ectopic beat immediate before the ectopic
                    all_peaks_before_ectopic_idx = np.where(rs_rpeaks_samples < ectopic)[0]
                    if len(all_peaks_before_ectopic_idx) > 0:
                        peak_before_ectopic_idx = all_peaks_before_ectopic_idx[-1]  # This is the index you want
                    else:
                        print(RED + 'ERROR: No peaks before ectopic beat' + RESET)
                        continue
                    
                    #define index of IBI containing ectopic beat (peak_before_ectopic_idx + 1)
                    ectopic_ibi_index = peak_before_ectopic_idx

                    # Create a cubic spline to interpolate the ibi in rs_rr_intervals at ectopic_ibi_index
                    
                    #Record original ibi value
                    original_rr_interval = rs_rr_intervals[ectopic_ibi_index]

                    # 3. Identify which RR intervals need interpolation
                    ectopic_rr_indices = []
                    for ep in idx_peaks_to_interpolate:
                        # Find flanking peaks in original array
                        before = np.where(rs_rpeaks_samples < ep)[0][-1]
                        after = before + 1
                        # Map to clean array indices
                        ectopic_rr_indices.extend([before-1, before])
                    
                    # 4. Use only valid intervals for spline fitting
                    valid_idx = np.setdiff1d(np.arange(len(rs_rr_intervals)), ectopic_rr_indices)
                    
                    if len(valid_idx) >= 4:
                        cs = CubicSpline(valid_idx, rs_rr_intervals[valid_idx])
                        interp_rr = cs(np.arange(len(rs_rr_intervals)))
                    else:
                        print(RED + 'ERROR: Not enough valid intervals for cubic spline interpolation' + RESET)
                        continue

                    #get interpolated value
                    interpolated_value = interp_rr[ectopic_ibi_index]

                    #print
                    print(BLUE + 'Interpolated IBI from', original_rr_interval, 'to', interpolated_value, RESET)
                    
                #convert rs_rr_intervals to sample numbers
                rs_rr_intervals_samples = np.concatenate(([0], np.cumsum(interp_rr)))  # Add the first peak at index 0

                return rs_rr_intervals_samples

            #If there are ectopic beats, then interpolate the RR intervals containing the ectopics
            if len(idx_subj_ectopics_to_interpolate) > 0:
                rpeaks_clean = interpolate_ectopic_rr_cubic_resting(
                rs_subj_ecg_peaks, rs_idx_peaks_to_delete)
            else:
                #Otherwise just use the original R-peak indices
                rpeaks_clean = rs_subj_ecg_peaks

            # 4. Calculate HRV metrics using neurokit2
            # neurokit2 expects R-peak indices (in samples) and the sampling rate
            subj_hrv_params = nk.hrv(rpeaks_clean, sampling_rate=SPIKE_FS, show=False)

            print('- Successfully calculated HRV parameters after ectopic interpolation')

            #Find subject rows in df_all_combined
            subj_rows_combined = df_all_combined['subjectID'].str.contains((subjID), na=False, case=False) #get logical array of rows containing participant ID

            # Calculate mean HR
            subj_hrv_params["HR_Mean"] = (60*SPIKE_FS)/subj_hrv_params["HRV_MeanNN"]
            
            # Select relevant HRV parameters
            hrv_metrics_to_keep = [
                "HR_Mean",     #Mean HR
                "HRV_MeanNN",  # Mean RR (NN) interval
                "HRV_SDNN",    # Standard deviation of NN intervals
                "HRV_RMSSD",   # Root mean square of successive differences
                "HRV_pNN50",   # Percentage of NN intervals >50ms
                "HRV_HF",      # High-frequency power (vagal tone)
                "HRV_LF",      # Low-frequency power
                "HRV_LFHF"     # LF/HF ratio
            ]

            # Create a dictionary to map original names to new names with 'rs_' prefix
            hrv_metrics_renamed = {metric: f"rs_{metric}" for metric in hrv_metrics_to_keep}

            # Ensure all required metrics exist in the HRV dataframe
            subj_hrv_params_filtered = subj_hrv_params[hrv_metrics_to_keep]

            # Assign HRV values to the corresponding subject rows in df_all_combined
            for original_metric, new_metric in hrv_metrics_renamed.items():
                df_all_combined.loc[subj_rows_combined, new_metric] = round(subj_hrv_params_filtered[original_metric].values[0], 5) #round to 5dp

                # exclude data from BG047c due to ventricular arrhythmia
                if subjID == 'BG047c': 
                    df_all_combined.loc[subj_rows_combined, new_metric] = pd.NA

            print('- Successfully added HRV parameters to df_all_combined with "rs_" prefix')

            #------------Process EDA data-------------------------------------------
            #----- Transform EDA signal (convert to microsiemens, filter low freq. drift ) 
            #Get eda data
            subj_eda_signal = subj_eda_data.magnitude.squeeze()
            subj_eda_signal = subj_eda_signal + 5 #add 5 to make all values positive -updated 15.04.25
            subj_eda_signal = subj_eda_signal * subj_transfer_constant #apply transfer constant to convert to microsiemens
            print('- Applied transfer constant ' + str(round(subj_transfer_constant, 2)))

            #get EDA signal between rs textmarks
            subj_eda_signal_rs = subj_eda_signal[int(tm_rs_start) : int(tm_rs_end)]

            #-------Plot EDA signal
            if eda_visual_verification:
                # Create time axis (in seconds)
                time = np.arange(len(subj_eda_signal_rs)) / 1000  # Convert to seconds

                # Plot with explicit time axis
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=time, y=subj_eda_signal_rs, mode='lines', name='Raw'))
                fig.show()

            #----Calculate SCL/EDA parameters
            subj_eda_signals_rs_nk, _ = nk.eda_process(subj_eda_signal_rs, sampling_rate=SPIKE_FS)
            subj_eda_interval_rs_nk = nk.eda_intervalrelated(subj_eda_signals_rs_nk, sampling_rate=SPIKE_FS)

            #Calculate sum SCR amplitudes
            subj_eda_interval_rs_nk['SCR_Peaks_Amplitude_Sum'] = subj_eda_interval_rs_nk['SCR_Peaks_Amplitude_Mean'] * subj_eda_interval_rs_nk['SCR_Peaks_N']
            
            #Calculate mean SCL
            subj_eda_interval_rs_nk['SCL_Mean'] = subj_eda_signals_rs_nk['EDA_Tonic'].mean()

            eda_metrics_to_keep = [
                "SCR_Peaks_N", # Number of SCR peaks
                "SCR_Peaks_Amplitude_Mean", #Mean SCR peak amplitude
                "SCR_Peaks_Amplitude_Sum", #Sum SCR peak amplitudes
                "SCL_Mean", #Mean SCL
                "EDA_Sympathetic" # Derived from Posada-Quintero et al. (2016), who argue that dynamics of the sympathetic component of EDA signal is represented in the frequency band of 0.045-0.25Hz.
            ]

            # Create a dictionary to map original names to new names with 'rs_' prefix
            eda_metrics_renamed = {metric: f"rs_{metric}" for metric in eda_metrics_to_keep}

            # Ensure all required metrics exist in the HRV dataframe
            subj_eda_params_filtered = subj_eda_interval_rs_nk[eda_metrics_to_keep]

            # Assign EDA values to the corresponding subject rows in df_all_combined
            for original_metric, new_metric in eda_metrics_renamed.items():
                df_all_combined.loc[subj_rows_combined, new_metric] = round(subj_eda_params_filtered[original_metric].values[0], 5) #round to 5dp

                # exclude data from BG007c due to faulty EDA signal
                if subjID == 'BG007c': 
                    df_all_combined.loc[subj_rows_combined, new_metric] = pd.NA
                    print(BLUE + '- BG007c rs EDA data excluded due to faulty recording' + RESET)

            print('- Successfully added EDA parameters to df_all_combined with "rs_" prefix')


Session:  BG001
[]
- using empathy.smrx spike file
- Importing Spike data for BG001
- p transfer constant = 2.5
- c transfer constant = 2.5
BG001p peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
BG001c peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
Session:  BG002
['F:/ADHD emotion/Empathy study/Participant data\\BG002\\BG002_rs.smrx']
- rs.smrx file found
- Importing Spike data for BG002
-

c:\Users\bgreenwood\AppData\Local\miniconda3\envs\empathy_env\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


- p transfer constant = 2.5
- c transfer constant = 2.5
BG007p peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
BG007c peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- BG007c rs EDA data excluded due to faulty recording
- BG007c rs EDA data excluded due to faulty recording
- BG007c rs EDA data excluded due to faulty recording
- BG007c rs EDA data excluded due to faulty recording
- BG007c rs EDA data excluded due to faulty recording
- Successfully added EDA parame

c:\Users\bgreenwood\AppData\Local\miniconda3\envs\empathy_env\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


BG017c peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
Session:  BG018
[]
- using empathy.smrx spike file
- Importing Spike data for BG018
- p transfer constant = 2.5
- c transfer constant = 2.5
BG018p peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
BG018c peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV par

...

- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
BG041c peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
Session:  BG042
['F:/ADHD emotion/Empathy study/Participant data\\BG042\\BG042_rs.smrx']
- rs.smrx file found
- Importing Spike data for BG042
- p transfer constant = 2.5
- c transfer constant = 2.5
BG042p peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully a

c:\Users\bgreenwood\AppData\Local\miniconda3\envs\empathy_env\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


- Successfully added EDA parameters to df_all_combined with "rs_" prefix
BG044c peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
Session:  BG045
['F:/ADHD emotion/Empathy study/Participant data\\BG045\\BG045_rs.smrx']
- rs.smrx file found
- Importing Spike data for BG045
- p transfer constant = 2.5
- c transfer constant = 2.5
BG045p peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
BG045c peak 

c:\Users\bgreenwood\AppData\Local\miniconda3\envs\empathy_env\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
BG047c peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully added HRV parameters to df_all_combined with "rs_" prefix
- Applied transfer constant 2.5
- Successfully added EDA parameters to df_all_combined with "rs_" prefix
Session:  BG048
['F:/ADHD emotion/Empathy study/Participant data\\BG048\\BG048_rs.smrx']
- rs.smrx file found
- Importing Spike data for BG048
- p transfer constant = 2.5
- c transfer constant = 2.5
BG048p peak deletion/addition
- No peaks deleted
idx_subj_ectopics_to_interpolate []
rs_idx_peaks_to_delete []
- Successfully calculated HRV parameters after ectopic interpolation
- Successfully a

Save data

In [ ]:
save_destination = BASE_DIR / "Participant data" / "Preprocessed task data" / "Empathy task" / 'df_all_combined.csv'

df_all_combined.to_csv(save_destination, index=False)  # Optional: index=False to avoid writing row indices


Function convert days to datetime

In [ ]:
from datetime import datetime, timedelta

def convert_days_to_datetime(days_since_2000):
    """
    Convert a number of days since 1 Jan 2000 into a formatted datetime string.

    This function converts a floating-point number representing days since 0 Jan 2000
    (with fractional part representing time of day) into a human-readable datetime string
    in the format "DD/MM/YY HHMMSS.mmm". The function adjusts the input so that
    day 1 corresponds to 1 Jan 2000.

    Parameters
    ----------
    days_since_2000 : float
        Number of days since 0 Jan 2000. The integer part represents whole days,
        and the fractional part represents the fraction of a 24-hour day.

    Returns
    -------
    str
        Datetime formatted as "DD/MM/YY HHMMSS.mmm" (e.g., "15/08/24 113015.123").

    Notes
    -----
    - Subtracts 1 from the input to align with 1 Jan 2000 as day 1.
    - Handles fractional days by converting to hours, minutes, seconds, and milliseconds.

    Example
    -------
    >>> days_since_2000 = 9021.48654358796
    >>> convert_days_to_datetime(days_since_2000)
    '28/08/24 115646.430'
    """
    
    # subtract 1 day to convert from 'days since 0 Jan 2000' to 'days since 1 Jan 2000'
    days_since_2000 = days_since_2000 - 1

    # Split the input into whole days and fractional part
    whole_days = int(days_since_2000)
    fractional_day = days_since_2000 - whole_days

    # Create a datetime object for Jan 1, 2000
    base_date = datetime(2000, 1, 1)

    # Add the number of days
    date = base_date + timedelta(days=whole_days)

    # Calculate hours, minutes, seconds, and milliseconds from the fractional day
    total_seconds = fractional_day * 24 * 3600
    hours = int(total_seconds / 3600)
    minutes = int((total_seconds % 3600) / 60)
    seconds = int(total_seconds % 60)
    milliseconds = int((total_seconds % 1) * 1000)

    # Combine date and time
    result_datetime = date.replace(hour=hours, minute=minutes, second=seconds, microsecond=milliseconds*1000)

    # Format the result
    formatted_time = result_datetime.strftime("%d/%m/%y %H%M%S.%f")[:-3]

    return formatted_time
